/storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/Imagenet2012

In [1]:
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["TORCH_LOGS"] = "output_code"
# os.environ["TORCH_LOGS"] = "inductor"
# os.environ["TORCHINDUCTOR_TRACE"] = "1"
# os.environ["TORCHINDUCTOR_VERBOSE"] = "1"
# os.environ["TORCHINDUCTOR_DEBUG"] = "1"
# os.environ["TORCHINDUCTOR_DUMP"] = "1"
# os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = "1"
# os.environ["TORCH_COMPILE_DEBUG"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (
    # "backend:cudaMallocAsync,"
    "expandable_segments:True,"
    # "garbage_collection_threshold:0.6"
)

from collections.abc import Iterable
import collections
collections.Iterable = Iterable
import webdataset as wds

import sys
sys.path.append('/home/hice1/yyu496/kaggle/CW')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch._functorch.aot_autograd import aot_module, make_boxed_func
from torch.autograd import grad
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import (
    LinearLR,
    CosineAnnealingLR,
    SequentialLR
)
from torch.optim.optimizer import Optimizer

import torch
from torch.profiler import profile, record_function, ProfilerActivity
import torch._dynamo as dynamo
import torch.nn.utils.parametrize as P
from torch.nn.utils.parametrizations import spectral_norm

import cpp_extension as cpp_extension

from controller import Controller
import cuda_graph_utils as cuda_utils
from layers import RMSNorm, DOConv2d, DOConv1d, DOLinear
from modules.activations.act_layers import DOReLU_Variance, DOSiLU, DOGELU
from fusion.fused_layers import DOBatchNormReLU2d
from freq_utils import radial_spectrum_2d, wasserstein, kl_div_spectrum
from ViT import VisionTransformerFreq, VisionTransformerFreqStrong
# from FreqViT import VisionTransformerFreqV2
from FreqViT import VisionTransformerFreqV4
# from VT_test import VisionTransformerV2, VisionTransformer
from VT_test import VisionTransformerV3

import timm
from timm.layers import trunc_normal_

import torchvision.models as models
from torchvision.transforms import v2

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.allow_tf32 = True

import random
import numpy as np
import copy

import triton

import actnn
# available choices are ["L0", "L1", "L2", "L3", "L4", "L5"]
actnn.set_optimization_level("L3")


from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchmetrics import Accuracy


import matplotlib.pyplot as plt
import math

/home/hice1/yyu496/.conda/envs/lib/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 512
warmup_epochs = 10
num_epochs = 512
# ============= Data ==================

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]


transform_train = v2.Compose([
    # --- Convert to tensor FIRST (kills PIL early) ---
    v2.ToImage(),                          # handles PIL → Tensor safely
    v2.ToDtype(torch.float32, scale=True),

    # --- Geometry ---
    v2.Resize((224, 224), antialias=True),
    # v2.Resize((112, 112), antialias=True),
    # v2.RandomResizedCrop(224, scale=(0.5, 1.0), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),

    # --- Color ---
    v2.RandomApply([
        v2.ColorJitter(
            brightness=0.2,
            contrast=0.2,
            saturation=0.2,
            hue=0.1
        )
    ], p=0.5),

    # --- RandAugment (vectorized, NO PIL) ---
    v2.RandAugment(num_ops=5, magnitude=9),

    # --- Normalize ---
    v2.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),

    # --- Random Erasing (tensor-based, fast) ---
    v2.RandomErasing(p=0.5, scale=(0.02, 0.33),
                     ratio=(0.3, 3.3), value=0),
])

check_transform = v2.Compose([
    v2.ToImage(),                          
    v2.ToDtype(torch.float32, scale=True),

    v2.Resize((224, 224), antialias=True),
    # v2.Resize((112, 112), antialias=True),

    v2.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])


#/home/hice1/yyu496/scratch/data/cifar10_resized/train
# /storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/Imagenet2012/train
#/storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/CIFAR-10-images
# check_set = ImageFolder(
#     root="/home/hice1/yyu496/scratch/data/cifar10_resized/train",
#     transform=check_transform
# )
check_set = datasets.CIFAR10(root="./data", train=True,
        download=True, transform=check_transform)

check_loder = DataLoader(
    check_set, batch_size=64, shuffle=True,
    num_workers=15, pin_memory=True, drop_last=True,
    persistent_workers=True, prefetch_factor=3
)

# download and prepare
train_dataset = datasets.CIFAR10(root="./data", train=True,
                                 download=True, transform=transform_train)
# train_dataset = ImageFolder(
#     root="/home/hice1/yyu496/scratch/data/cifar10_resized/train",
#     transform=transform_train
# )

# DataLoader with GPU-friendly settings
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=15, pin_memory=True, drop_last=True,
                          persistent_workers=True, prefetch_factor=3)

TRAIN_PATH = "/home/hice1/yyu496/scratch/data/cifar10_shards/train/train-{000000..000009}.tar"
VAL_PATH   = "/home/hice1/yyu496/scratch/data/cifar10_shards/val/val-{000000..000001}.tar"


# trainset = (
#     wds.WebDataset(wds.ResampledShards(TRAIN_PATH), empty_check=False)
#     .shuffle(256)                          # small buffer is enough for CIFAR
#     .decode("torch")
#     .to_tuple("png", "cls")
#     .batched(batch_size, partial=False)    # batch FIRST
#     .map(lambda b: (transform_train(b[0]), b[1]))   # transform the whole batch
# )

# train_loader = (
#     wds.WebLoader(
#         trainset,
#         batch_size=None,                   # already batched
#         num_workers=2,                     # CIFAR: 2–4 is plenty
#         pin_memory=True,
#         persistent_workers=False,          # IMPORTANT for RSS not to accumulate
#         prefetch_factor=2,
#     )
#     .with_epoch(50000 // batch_size)
# )



val_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),

    v2.Resize((224, 224), antialias=True),
    # v2.Resize((112, 112), antialias=True),

    v2.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

# /storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/Imagenet2012/val
# /storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/CIFAR-10-images/test
#/home/hice1/yyu496/scratch/data/cifar10_resized/test
# === CIFAR-10 validation dataset (train=False) ===
val_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=val_transform
)
# val_dataset = ImageFolder(
#     root="/home/hice1/yyu496/scratch/data/cifar10_resized/test",
#     transform=val_transform
# )

# === Validation DataLoader ===
val_loader = DataLoader(
    val_dataset,
    batch_size=512,
    shuffle=False,   
    num_workers=15,
    pin_memory=True,
    persistent_workers=True, 
    prefetch_factor=3
)
# valset = (
#     wds.WebDataset(VAL_PATH, shardshuffle=False, empty_check=False)
#     .decode("torch")
#     .to_tuple("png", "cls")
#     .batched(128, partial=False)
#     .map(lambda b: (val_transform(b[0]), b[1]))
# )

# val_loader = (
#     wds.WebLoader(
#         valset,
#         batch_size=None,
#         num_workers=1,
#         pin_memory=True,
#         persistent_workers=False,
#         prefetch_factor=2,

#     )
#     .with_epoch(10000 // 128)
# )



def mixup_cutmix(x, y, alpha=1.0, p_cutmix=0.5):
    if alpha <= 0:
        return x, y

    lam = torch.distributions.Beta(alpha, alpha).sample().to(x.device)

    b = x.size(0)
    perm = torch.randperm(b, device=x.device)
    x2, y2 = x[perm], y[perm]

    H, W = x.shape[-2:]

    if torch.rand(()) < p_cutmix:
        # ---- CutMix ----
        cx = torch.randint(W, (1,), device=x.device)
        cy = torch.randint(H, (1,), device=x.device)

        w = (W * torch.sqrt(1 - lam)).long()
        h = (H * torch.sqrt(1 - lam)).long()

        x1 = torch.clamp(cx - w // 2, 0, W)
        x2b = torch.clamp(cx + w // 2, 0, W)
        y1 = torch.clamp(cy - h // 2, 0, H)
        y2b = torch.clamp(cy + h // 2, 0, H)

        x[:, :, y1:y2b, x1:x2b] = x2[:, :, y1:y2b, x1:x2b]
        lam = 1 - ((x2b-x1)*(y2b-y1) / (W*H))
    else:
        # ---- Mixup ----
        x = lam * x + (1 - lam) * x2

    return x, (y, y2, lam)

def replace_activation_func(m):
    for name, child in m.named_children():
        if isinstance(child, nn.ReLU):
            setattr(m, name, nn.SiLU(inplace=True))
            # setattr(m, name, nn.ReLU6(inplace=True))
            # setattr(m, name, nn.LeakyReLU(inplace=True))
            # setattr(m, name, nn.GELU())
        else:
            replace_activation_func(child)


def replace_norm(m):
    for name, child in m.named_children():
        if isinstance(child, nn.LayerNorm):
            setattr(m, name, RMSNorm(dims=child.normalized_shape[-1]))
        else:
            replace_activation_func(child)


def disable_act_inplace(m):
    for module in m.modules():
        if isinstance(module, (nn.ReLU, nn.SiLU, nn.GELU, nn.ReLU6)) and hasattr(module, 'inplace'):
            module.inplace = False

def set_seed(SEED):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

def init_fan_out(m):
    if isinstance(m, (nn.Conv2d,)):
        nn.init.kaiming_normal_(
            m.weight,
            mode='fan_out',
            nonlinearity='relu'
        )
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(
            m.weight,
            mode='fan_in',
            nonlinearity='relu'
        )

        if m.bias is not None:
            nn.init.zeros_(m.bias)


def apply_trunc_init_for_act(module, std=0.02):
    """
    Truncated normal init tuned for ACT stability.
    Applies only to ACT-sensitive layers.
    """

    # ---- Convs (ACT-fragile entry + transition layers) ----
    if isinstance(module, nn.Conv2d):
        # stem conv or downsample conv benefit most
        if module.kernel_size[0] > 1 or module.in_channels < module.out_channels:
            trunc_normal_(module.weight, std=std)
        else:
            nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')

        if module.bias is not None:
            nn.init.zeros_(module.bias)

    # ---- Linear layers (ViT / MLP / projections) ----
    elif isinstance(module, nn.Linear):
        trunc_normal_(module.weight, std=std)
        if module.bias is not None:
            nn.init.zeros_(module.bias)

    # ---- BatchNorm / LayerNorm ----
    elif isinstance(module, (nn.BatchNorm2d, nn.GroupNorm, nn.LayerNorm)):
        nn.init.ones_(module.weight)
        nn.init.zeros_(module.bias)



def init_orthogonal_conv(m, gain=1.0):
    if isinstance(m, nn.Conv2d):
        nn.init.orthogonal_(m.weight, gain=gain)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

    elif isinstance(m, nn.Linear):
        # nn.init.orthogonal_(m.weight, gain=gain)
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)


def init_orthogonal_conv_v2(n, m, gain=1.0):
    if isinstance(m, nn.Conv2d):
        nn.init.orthogonal_(m.weight, gain=gain)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    if n == 'layer2.0.conv1' and isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)
        print('True')

    elif isinstance(m, nn.Linear):
        # nn.init.orthogonal_(m.weight, gain=gain)
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)


In [3]:
act_cache = {}
def forward_hook(name):
    def hook(module, input, output):
        global act_cache
        act_cache[name] = output
    return hook

In [4]:
# DIVISION = {
#     'pool_kernel_size' : 3
# }

DIVISION = None

config = {
    "default_bits": 2,
    'analyze' : False,
    'auto_precision': None,
    'DIVISION' : DIVISION,
    'AVG_ALAM' : False,
    "AVG_ALAM_BTS" : 4,
    "group_size": 256,
    'batch_size' : batch_size,
    'fp8' : False,
    'depth_point_conv' : False,
    'rms_norm' : False
}

# vit_base_patch16_224
# efficientnet_b0
num_classes = 10
# model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=num_classes)
# replace_activation_func(model)
# replace_norm(model)
# model = VisionTransformer(
#     image_size=224,
#     patch_size=16,
#     in_channels=3,
#     num_classes=num_classes,
#     embed_dim=384,
#     depth=8,
#     num_head=6,
#     mlp_ratio=4.0,
#     attn_p=0.2,
#     mlp_p=0.0,
#     pos_p=0.0
# ).cuda()

# model = VisionTransformerV2(
#     image_size=224,
#     patch_size=16,
#     in_channels=3,
#     num_classes=num_classes,
#     embed_dim=384,
#     depth=8,
#     num_head=6,
#     mlp_ratio=4.0,
#     attn_p=0.2,
#     mlp_p=0.0,
#     pos_p=0.0
# ).cuda()

model = VisionTransformerV3(
    image_size=224,
    patch_size=16,
    in_channels=3,
    num_classes=num_classes,
    embed_dim=384,
    depth=8,
    num_head=6,
    mlp_ratio=4.0,
    attn_p=0.2,
    mlp_p=0.0,
    pos_p=0.0
).cuda()

# model = VisionTransformerFreq(
#         in_channels=3, num_classes=10,
#         image_size=224, patch_size=16,
#         embed_dim=384, depth=8, num_head=6, mlp_ratio=4.0,
#         attn_p=0.2, mlp_p=0.0, pos_p=0.0,
#         num_freq_bins=16,
#         use_proxy_head=False,
#         inject_every_block=False,
#         injector_init_scale=1.0,
#     ).cuda()

# model = VisionTransformerFreqV2(
#     in_channels=3,
#     num_classes=10,
#     image_size=224,
#     patch_size=16,
#     embed_dim=384,
#     depth=12,
#     num_head=6,
#     freq_bins=16,
#     inject_every_block=True,
#     injector_scale=0.05,
#     use_proxy_head=False,
# )
# model = VisionTransformerFreqV2(
#         in_channels=3,
#         num_classes=10,
#         image_size=224,
#         patch_size=16,
#         embed_dim=384,
#         depth=12,
#         num_head=6,
#         num_bands=6,
#         inject_patchwise_bias=True,
#         anchor_freq_token=True,
#     )
# model = VisionTransformerFreqStrong(
#     in_channels = 3,
#     num_classes = 10,
#     image_size = 224,
#     patch_size = 16,
#     embed_dim = 768,
#     depth = 12,
#     num_head = 12,
#     mlp_ratio = 4.0,
#     attn_p = 0.0,
#     mlp_p = 0.0,
#     pos_p = 0.0,
#     num_bands = 8,
#     retain_scale = 0.5,
#     anchor_freq_token = True,
#     anchor_weight = 0.04,
# )


# model = VisionTransformerFreqV3(
#         in_channels=3,
#         num_classes=10,
#         image_size=224,
#         patch_size=16,
#         embed_dim=384,
#         depth=8,
#         num_head=6,
#         mlp_ratio=4.0,
#         attn_p=0.2,
#         proxy_channels=8,
#         radial_bins=4,
#         angular_bins=4,
#         alpha=0.10,
#         detach_freq_token_update=True,
#     )
# model = VisionTransformerFreqV4(
#         in_channels = 3,
#         num_classes = num_classes,
#         image_size = 224,
#         patch_size = 16,
#         embed_dim = 384,
#         depth = 8,
#         num_head = 6,
#         mlp_ratio = 4.0,
#         attn_p = 0.0,
#         mlp_p = 0.0,
#         pos_p = 0.0,
#         proxy_channels = 8,
#         radial_bins = 4,
#         angular_bins = 4,
#         detach_freq_token_update = True,
#         detach_spec_from_patch = True,
#         alpha = 0.10,
# )



# set_seed(2)
# model = models.resnet44(weights=None)
# model = models.resnet50(weights=None)
# model = models.resnet18(weights=None)
# model = models.efficientnet_b0(weights=None)
# model = models.mobilenet_v2(weights=None)
# model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
# model.fc = nn.Linear(model.fc.in_features, num_classes)
# replace_activation_func(model)
disable_act_inplace(model)
# model.apply(lambda m: apply_trunc_init_for_act(m, std=0.02))
# model.apply(init_fan_out)
# model.apply(lambda m: init_orthogonal_conv(m, gain=math.sqrt(2)))
# init(model)
# model.load_state_dict(torch.load("/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192_5.pth"))


# model = actnn.QModule(model)
# model.cuda()
# model.compile(fullgraph=True)


# set_seed(49)
# torch.set_rng_state(cached_rng_state['CPU'])
# torch.cuda.set_rng_state(cached_rng_state['GPU'])
# model.load_state_dict(baseline)
# disable_act_inplace(model)
# # init(model)
# for n, m in model.named_modules():
#     if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Linear)):
#             cahced_init(m.weight)



criterion = nn.CrossEntropyLoss()

# compute_stream = torch.cuda.Stream()
# controller = Controller(model, config, check_loder, criterion, test=False)
# controller.iterate(criterion)
# controller.warp_model(graph_mode=True, quantizer=True)
# controller.warmup(train_loader, criterion, compute_stream)

# blocks.7.mlp.depthconv1
# blocks.7.mlp.pointconv2
# patch_embed.proj
# blocks.11.mlp.fc2
# layer4.2.conv3
# layer4.1.conv2
# features.8.0
# features.18.0
#'features.8.0' in name or "features.6.2.block.0.0" in name or "features.5.1.block.0.0" in

# for name, module in controller.traced_model.named_modules():
#     if isinstance(module, (DOConv2d, DOConv1d, DOLinear)):
#         if 'blocks.7.mlp.pointconv2' in name:
#             m = module.register_forward_hook(forward_hook(name))
#             print(module)

for name, module in model.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Conv1d)):
        if 'blocks.7.mlp.depthconv1' in name:
            m = module.register_forward_hook(forward_hook(name))
            print(module)

Conv2d(1536, 1536, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=1536)


In [5]:
# num_params = sum(p.numel() for p in controller.traced_model.parameters())
# print(f"Total params: {num_params/1e6:.2f} M")

num_params = sum(p.numel() for p in model.parameters())
print(f"Total params: {num_params/1e6:.2f} M")

Total params: 14.70 M


In [6]:
class SGD_LARS(Optimizer):
    def __init__(self,
                 params,
                 lr=0.1,
                 momentum=0.9,
                 weight_decay=1e-4,
                 eta=0.1,
                 eps=1e-9,
                 nesterov=False,
                 trust_coef=True):
        defaults = dict(lr=lr,
                        momentum=momentum,
                        weight_decay=weight_decay,
                        eta=eta,
                        eps=eps,
                        nesterov=nesterov,
                        trust_coef=trust_coef)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            lr = group['lr']
            momentum = group['momentum']
            weight_decay = group['weight_decay']
            eta = group['eta']
            eps = group['eps']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue

                grad = p.grad

                # ---- decoupled weight decay ----
                if weight_decay != 0 and p.ndim > 1:
                    grad = grad.add(p, alpha=weight_decay)

                # ---- LARS trust ratio (only apply to weight tensors) ----
                if p.ndim > 1:      # don't LARS biases / norms
                    w_norm = p.norm()
                    g_norm = grad.norm()
                    trust_ratio = 0

                    # if not torch.isfinite(grad).all():
                    #     print("grad has NaN/Inf")
                    # if not torch.isfinite(w_norm):
                    #     print("w_norm NaN/Inf:", w_norm.item())
                    # if not torch.isfinite(g_norm):
                    #     print("g_norm NaN/Inf:", g_norm.item())

                    if w_norm > 0 and g_norm > 0:
                        trust_ratio = eta * (w_norm / (g_norm + eps))
                    else:
                        trust_ratio = 1.0

                    # if not torch.isfinite(trust_ratio):
                    #     print("trust_ratio NaN/Inf:", trust_ratio.item())
                    # print("trust_ratio", trust_ratio.item(), "dw_norm", g_norm.item(), "w_norm", w_norm.item())

                    
                else:
                    trust_ratio = 1.0

                scaled_lr = lr * trust_ratio

                # ---- Momentum buffer ----
                param_state = self.state[p]
                if 'momentum_buffer' not in param_state:
                    buf = param_state['momentum_buffer'] = torch.zeros_like(p)
                else:
                    buf = param_state['momentum_buffer']

                buf.mul_(momentum).add_(grad)  # standard momentum update

                if nesterov:
                    update = grad.add(buf, alpha=momentum)
                else:
                    update = buf

                # ---- Update weights ----
                p.add_(update, alpha=-scaled_lr)

        



In [7]:
acc = Accuracy(task='multiclass', num_classes=10, average='macro')

# opt = torch.optim.SGD([torch.tensor([0.], device='cuda', requires_grad=True)], lr=1e-2 * 5)

# opt = torch.optim.AdamW(controller.traced_model.parameters(), lr=(1e-5 * 8), weight_decay=1e-4, fused=True, capturable=True)
# opt.load_state_dict(torch.load('/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192_opt_6.pth'))
# opt = SGD_LARS(
#     controller.traced_model.parameters(),
#     lr=1e-3 * 8,                     # LARS needs high LR
#     momentum=0.9,
#     weight_decay=1e-7,          # decoupled weight decay
#     eta=0.1,                  # LARS coefficient (0.001–0.02)
#     nesterov=True
# )
opt = torch.optim.AdamW(model.parameters(), lr=(1e-5 * 8), fused=True, capturable=True)
# opt = SGD_LARS(
#     model.parameters(),
#     lr=1.0,                     # LARS needs high LR
#     momentum=0.9,
#     weight_decay=1e-7,          # decoupled weight decay
#     eta=0.001,                  # LARS coefficient (0.001–0.02)
#     nesterov=False
# )


total_timer_start = torch.cuda.Event(enable_timing=True)
total_timer_end = torch.cuda.Event(enable_timing=True)
total_time = 0.0

e_timer_start = torch.cuda.Event(enable_timing=True)
e_timer_end = torch.cuda.Event(enable_timing=True)
partile_time = 0.0


# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     opt,
#     1e-3,
#     total_steps= num_epochs * len(train_loader),
#     pct_start=0.1,
#     div_factor=10,
#     final_div_factor=1000
# )

# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#     opt,
#     T_0=10,
#     T_mult=2,
#     eta_min=1e-5
# )

warmup_sched = LinearLR(
    opt,
    start_factor=0.5,   # lr = lr * start_factor
    end_factor=1.0,    # lr = lr * end_factor = lr
    total_iters=warmup_epochs
)

# 2. Cosine decay
cosine_sched = CosineAnnealingLR(
    opt,
    T_max=num_epochs - warmup_epochs,
    eta_min=1e-6
)

# 3. Combine them
scheduler = SequentialLR(
    opt,
    schedulers=[warmup_sched, cosine_sched],
    milestones=[warmup_epochs]
)

cuda_graph_generator = cuda_utils.Graph(controller.traced_model,
                                        criterion,
                                        opt,
                                        train_loader,
                                        compute_stream,
                                        mode='qdrop',
                                        num_of_graph=1,
                                        device='cuda')


# cuda_graph_generator = cuda_utils.Graph(model,
#                                         criterion,
#                                         opt,
#                                         train_loader,
#                                         compute_stream,
#                                         mode='qdrop',
#                                         num_of_graph=1,
#                                         device='cuda')

graphs, compute_stream, static_x, static_y, logits, loss = cuda_graph_generator.capture_cuda_graph_qdrop()

total_graphs = cuda_graph_generator.num_of_graph

In [8]:
@torch.no_grad()
def lowpass_param_2d(t: torch.Tensor, k: int = 3):
    if t.ndim != 4:
        return t

    *prefix, H, W = t.shape
    if H < k or W < k:
        return t  # nothing to do, kernel larger than spatial size

    # Merge prefix dims into batch*channels, so shape -> [N, 1, H, W]
    t_flat = t.view(-1, 1, H, W)
    t_l = F.avg_pool2d(t_flat, kernel_size=k, stride=1, padding=k // 2)
    t_l = F.interpolate(t_l, size=(H, W), mode="bilinear", align_corners=False)

    return t_l.view_as(t)


@torch.no_grad()
def inject_grad_noise_large_batch(
    model,
    step,
    batch_size,
    total_samples,
    base_std=1,
    gamma=0.55,
    lp_kernel: int = 3
):
    # Large-batch SGD noise recovery term
    gns_scale = (1.0 / batch_size - 1.0 / total_samples) ** 0.5
    decay = 1.0 / ((1 + step) ** gamma)

    for p in model.parameters():
        if p.grad is None:
            continue

        # Skip bias / norm (critical for ViT & BN stability)
        if p.ndim == 1:
            continue


        grad_std = p.grad.std().clamp(min=1e-6)

        noise = torch.randn_like(p.grad) * base_std * gns_scale * grad_std * decay

        if lp_kernel is not None and lp_kernel > 1:
            noise = lowpass_param_2d(noise, k=lp_kernel)

        p.grad.add_(noise)


def update_lr(model, scheduler):
    lr = scheduler.get_last_lr()[0]
    for m in model.modules():
        if hasattr(m, 'ema_grad_meta'):
            m.ema_grad_meta['lr'].copy_(lr)


def checkpoint_save_helper(model, acc, max_acc, opt):
    if acc >= max_acc:
        max_acc = acc
        torch.save(model.state_dict(), "/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192_6.pth")
        torch.save(opt.state_dict(), '/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192_opt_6.pth')
        return max_acc
    return max_acc


def freq_weight(step, warmup_steps=5000, max_w=0.1):
    if step >= warmup_steps:
        return max_w
    t = step / warmup_steps
    return max_w * t


def debug_check_grads_and_step(model, optimizer):
    # 1) check any grad exists
    gsum = 0.0
    gmax = 0.0
    n = 0
    for p in model.parameters():
        if p.grad is None:
            continue
        gn = p.grad.detach().float().norm().item()
        gsum += gn
        gmax = max(gmax, gn)
        n += 1
    print(f"[debug] grad tensors: {n}, grad_norm_sum: {gsum:.4e}, grad_norm_max: {gmax:.4e}")

    # 2) check that optimizer actually changes weights
    p0 = None
    for p in model.parameters():
        if p.requires_grad:
            p0 = p.detach().clone()
            p_ref = p
            break
    if p0 is None:
        print("[debug] no trainable parameter found!")
        return

    optimizer.step()

    delta = (p0 - p_ref.detach()).abs().mean().item()
    print(f"[debug] mean |Δparam| after step: {delta:.4e}")


In [9]:
torch.cuda.empty_cache()

G_GNS = []
G_SNR = []


def gradient_noise_scale(grad):
    tr_sigma = grad.var(0, unbiased=False).sum()
    return tr_sigma / (grad.mean(0).pow(2).sum() + 1e-12)

def signal_to_noise_ratio(grad):
    mu = grad.mean(0)
    tr_sigma = grad.var(0, unbiased=False).sum()
    return mu.norm() / (tr_sigma.sqrt() + 1e-12)

max_acc = 0.0
global_step = 0
torch.cuda.synchronize()
total_timer_start.record()
for i in range(num_epochs):
    train_logits = []
    train_y = []
    

    val_logits = []
    val_y = []
    partile_time = 0
    # controller.traced_model.train()
    model.train()
    # aot_model.train()
    G = []
    torch.cuda.synchronize()
    # k = i % total_graphs
    for step, (x, y) in enumerate(train_loader):
        
        # torch.cuda.current_stream().wait_stream(warmup)  
        x, y = x.to('cuda', non_blocking=True), y.to('cuda', non_blocking=True)
        
        e_timer_start.record()
        # compute_stream.wait_stream(torch.cuda.current_stream())
        # with torch.cuda.stream(compute_stream):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=True):
                # static_x.copy_(x)
                # static_y.copy_(y)
                # opt.zero_grad(set_to_none=False)

                # total_steps = i * len(train_loader) + step
                # k = step % total_graphs
                # add_smoothout_noise(controller.traced_model)
                # assign_theta(controller.traced_model)
                # graphs[0].replay()
                # remove_smoothout_noise(controller.traced_model)
                # opt.step()
                # scheduler.step()   
                
                # train_logits.append(logits.detach().cpu())
                # train_y.append(static_y.detach().cpu())  

                # for m in controller.traced_model.modules():
                #     if hasattr(m, "qdrop"):
                #         temp = torch.rand(1).item()
                #         m.qdrop.copy_(temp)

            opt.zero_grad()
            # raw_logits = controller.traced_model(x)
            raw_logits = model(x)

            if isinstance(raw_logits, tuple):
                logits = raw_logits[0]
                freq_maps = raw_logits[1]
            else:
                logits = raw_logits
                freq_maps = None

            CE_loss = criterion(logits, y)

            # 'features.8.0' in name or 'features.1.0.block.2.0'
            # 'features.8.0' in name or "features.6.2.block.0.0" in name or "features.5.1.block.0.0"
            model_spectrum1 = radial_spectrum_2d(act_cache['blocks.7.mlp.depthconv1'], num_rad_bins=16, num_ang_bins=1, mode='magnitude')
            # model_spectrum1 = radial_spectrum_2d(freq_maps["freq_token"], num_rad_bins=16, num_ang_bins=8)
            # model_spectrum2 = radial_spectrum_2d(freq_maps["patch_token"], num_rad_bins=16, num_ang_bins=8)
            # model_spectrum1 = radial_spectrum_2d(freq_maps, num_bins=8)
            # model_spectrum2 = radial_spectrum_2d(act_cache['features.1.0.block.2.0'], num_bins=32)
            # model_spectrum3 = radial_spectrum_2d(act_cache['features.5.1.block.0.0'], num_bins=16)
            target_spectrum = radial_spectrum_2d(x, num_rad_bins=16, num_ang_bins=1, mode='magnitude')
            # target_spectrum2 = radial_spectrum_2d(x, num_rad_bins=32, num_ang_bins=32)
            # model_spectrum = radial_spectrum_2d_channelwise_dct(act, num_bins=16)
            # target_spectrum = radial_spectrum_2d_channelwise_dct(x, num_bins=16).detach()

            freq_loss1 = wasserstein(model_spectrum1, target_spectrum)
            # freq_loss2 = wasserstein_1d(model_spectrum2, target_spectrum)
            # freq_loss3 = wasserstein_1d(model_spectrum3, target_spectrum)
            # freq_loss1 = kl_div_spectrum(target_spectrum, model_spectrum1)
            # freq_loss = temp_kl(target_spectrum, model_spectrum)
            # freq_loss = vit_spectral_slope_loss(act)
            # freq_loss = local_freq_kl_loss(act, x)
            # freq_loss = act_freq_loss(act, x, num_bins=8)
            # freq_loss1 = freq_maps["anchor_loss"]
            # freq_loss1 = freq_maps.get("anchor_loss",0) + freq_maps.get("usage_reg",0)

            # freq_loss = polar_wasserstein_rtheta(model_spectrum, target_spectrum)
            
            # global_step += 1
            # w = freq_weight(global_step, warmup_steps=100, max_w=1.0)

            # cache = PolarBinCache()

            # polar_s = polar_spectrum_2d(
            #     act, num_bins=32, num_theta_bins=64, remove_dc=True, power=True, bin_cache=cache
            # )
            # polar_t = polar_spectrum_2d(
            #     x, num_bins=32, num_theta_bins=64, remove_dc=True, power=True, bin_cache=cache
            # )

            # freq_loss = polar_kl_loss(polar_s, polar_t, T=1.5, k_low=12)       # strong + fast
            # freq_loss = lf_hf_logratio_loss(polar_s, polar_t, k_low=12)

            w = 1.0
            # w = w * (CE_loss.detach() / (freq_loss.detach() + 1e-8))  # keep terms balanced
            loss = CE_loss + w * freq_loss1
            # freq_loss.backward()
            # loss = CE_loss
            # loss = freq_loss1
            

            # debug_check_grads_and_step(controller.traced_model, opt)
            # inject_grad_noise_large_batch(controller.traced_model, step, batch_size, len(train_loader))
            # inject_grad_noise_large_batch(model, step, batch_size, len(train_loader))
            loss.backward()
            opt.step()
            # scheduler.step()     
            
            # if step % 20 == 0:
            #     temp = torch.nn.utils.parameters_to_vector(p.reshape(-1) for p in controller.traced_model.parameters() if p.grad is not None)
            #     # temp = torch.nn.utils.parameters_to_vector(p.reshape(-1) for p in model.parameters() if p.grad is not None)
            #     G.append(temp)     
                
            e_timer_end.record()
            torch.cuda.synchronize()
            partile_time += e_timer_start.elapsed_time(e_timer_end)

            train_logits.append(logits.detach().cpu())
            train_y.append(y.detach().cpu())

                # print("train logits mean:", logits.mean().item())
                # print("train logits std:", logits.std().item())

    # partile_time = e_timer_start.elapsed_time(e_timer_end)
    # total_time += e_timer_start.elapsed_time(e_timer_end)


    train_logits = torch.cat(train_logits)
    train_y = torch.cat(train_y)
    computed_acc = acc(train_logits, train_y)
    throughtout = 50000 / (partile_time / 1000)

    # temp_G = torch.cat(G)
    # GNS = gradient_noise_scale(temp_G)
    # SNR = signal_to_noise_ratio(temp_G)

    print(f'Epoch: {i}')
    print(f"Train Loss: {loss}")
    print(f"CE Loss: {CE_loss.item()}, Freq(freq) Loss1: {freq_loss1.item() * w}")
    # print(f"CE Loss: {CE_loss.item()}")
    print(f"Learning Rate: {scheduler.get_last_lr()[0]}")
    print(f"Train Accuracy: {computed_acc}")
    print(f'Peak Mem Reserved: {torch.cuda.max_memory_reserved()}')
    print(f'Peak Mem Allocated: {torch.cuda.max_memory_allocated()}')
    print(f'Current train time: {partile_time / 1000} s')
    print(f"Throughout: {throughtout} samples per second")
    # for name, m in controller.traced_model.named_modules():
    #     if isinstance(m, DOBatchNormReLU2d):
    #         if not torch.isfinite(m.running_mean).all() or not torch.isfinite(m.running_var).all():
    #             print("BAD BN:", name)
    #             print("mean finite:", torch.isfinite(m.running_mean).all().item(),
    #                 "var finite:", torch.isfinite(m.running_var).all().item(),
    #                 "var min:", m.running_var.min().item())
    #             break
    # print(f'GNS: {GNS}')
    # print(f'SNR: {SNR}')
    # G_GNS.append(GNS)
    # G_SNR.append(SNR)
    scheduler.step()
    # update_lr(controller.traced_model, scheduler)

    train_logits = []
    train_y = []

    # controller.traced_model.eval()
    model.eval()
    # aot_model.eval()
    with torch.compiler.set_stance("force_eager"):
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to('cuda'), y_val.to('cuda')

                with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=True):
                    # raw_y_preds  = controller.traced_model(x_val)
                    raw_y_preds = model(x_val)

                    if isinstance(raw_y_preds, tuple):
                        y_preds = raw_y_preds[0]
                    else:
                        y_preds = raw_y_preds

                val_loss = F.cross_entropy(y_preds, y_val)

                val_logits.append(y_preds.detach().cpu())
                val_y.append(y_val.detach().cpu())

    val_logits = torch.cat(val_logits)
    val_y = torch.cat(val_y)
    

    valid_computed_acc = acc(val_logits, val_y)
    # max_acc = checkpoint_save_helper(controller.traced_model, valid_computed_acc, max_acc, opt)
    print(f"Val Loss: {val_loss}")
    print(f"Val Accuracy: {valid_computed_acc}\n\n")


total_timer_end.record()
torch.cuda.synchronize()


full_time = total_timer_start.elapsed_time(total_timer_end)

Epoch: 0
Train Loss: 3.0790367126464844
CE Loss: 2.0988311767578125, Freq(freq) Loss1: 0.9802054166793823
Learning Rate: 4e-05
Train Accuracy: 0.20214886963367462
Peak Mem Reserved: 23662166016
Peak Mem Allocated: 21414108160
Current train time: 27.949401397705078 s
Throughout: 1788.94707934981 samples per second
Val Loss: 1.7890625
Val Accuracy: 0.30869999527931213


Epoch: 1
Train Loss: 2.819697141647339
CE Loss: 1.8394660949707031, Freq(freq) Loss1: 0.9802310466766357
Learning Rate: 4.4000000000000006e-05
Train Accuracy: 0.2716792821884155
Peak Mem Reserved: 23662166016
Peak Mem Allocated: 21745843712
Current train time: 20.75529783630371 s
Throughout: 2409.0234885737705 samples per second
Val Loss: 1.703125
Val Accuracy: 0.37560001015663147


Epoch: 2
Train Loss: 2.819685935974121
CE Loss: 1.840576171875, Freq(freq) Loss1: 0.9791098833084106
Learning Rate: 4.8e-05
Train Accuracy: 0.3106737732887268
Peak Mem Reserved: 23769120768
Peak Mem Allocated: 21745843712
Current train time: 2

/home/hice1/yyu496/.conda/envs/lib/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Val Loss: 1.2578125
Val Accuracy: 0.5562000274658203


Epoch: 10
Train Loss: 2.4405410289764404
CE Loss: 1.4619922637939453, Freq(freq) Loss1: 0.9785487651824951
Learning Rate: 8e-05
Train Accuracy: 0.46341997385025024
Peak Mem Reserved: 24062722048
Peak Mem Allocated: 21745843712
Current train time: 20.743200119018553 s
Throughout: 2410.428463935858 samples per second
Val Loss: 1.1640625
Val Accuracy: 0.5642000436782837


Epoch: 11
Train Loss: 2.4384372234344482
CE Loss: 1.4610824584960938, Freq(freq) Loss1: 0.9773548245429993
Learning Rate: 7.999922650413984e-05
Train Accuracy: 0.4738887548446655
Peak Mem Reserved: 24062722048
Peak Mem Allocated: 21745843712
Current train time: 20.740986038208007 s
Throughout: 2410.6857749141 samples per second
Val Loss: 1.1328125
Val Accuracy: 0.5961999893188477


Epoch: 12
Train Loss: 2.392547130584717
CE Loss: 1.4130148887634277, Freq(freq) Loss1: 0.9795323014259338
Learning Rate: 7.99969060468528e-05
Train Accuracy: 0.4813571572303772
Peak Mem Re

In [10]:
Epoch: 0
Train Loss: 2.1261978149414062
CE Loss: 2.0997085571289062, Freq Loss: 0.0264892578125
Learning Rate: 0.0004
Train Accuracy: 0.15813705325126648
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 6875020800
Current train time: 63.0915924987793 s
Throughout: 792.4986201761733 samples per second
Val Loss: 2.0625
Val Accuracy: 0.23239998519420624


Epoch: 1
Train Loss: 2.038616180419922
CE Loss: 2.016033172607422, Freq Loss: 0.0225830078125
Learning Rate: 0.00041600000000000003
Train Accuracy: 0.2195732593536377
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 7038803968
Current train time: 44.206503173828125 s
Throughout: 1131.0553065776494 samples per second
Val Loss: 1.765625
Val Accuracy: 0.31839999556541443


Epoch: 2
Train Loss: 1.939453125
CE Loss: 1.9161376953125, Freq Loss: 0.0233154296875
Learning Rate: 0.0004320000000000001
Train Accuracy: 0.26298731565475464
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 7038803968
Current train time: 44.21969369506836 s
Throughout: 1130.7179182377804 samples per second
Val Loss: 1.625
Val Accuracy: 0.38350003957748413


Epoch: 3
Train Loss: 1.8823909759521484
CE Loss: 1.8601741790771484, Freq Loss: 0.022216796875
Learning Rate: 0.00044800000000000005
Train Accuracy: 0.3026961088180542
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 7038803968
Current train time: 44.10707327270508 s
Throughout: 1133.6050272676255 samples per second
Val Loss: 1.5078125
Val Accuracy: 0.4239000082015991


Epoch: 4
Train Loss: 1.8371562957763672
CE Loss: 1.8153057098388672, Freq Loss: 0.0218505859375
Learning Rate: 0.0004640000000000001
Train Accuracy: 0.3301789462566376
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 7038803968
Current train time: 45.0461745300293 s
Throughout: 1109.9721679288953 samples per second
Val Loss: 1.4609375
Val Accuracy: 0.45670002698898315


Epoch: 5
Train Loss: 1.7238731384277344
CE Loss: 1.7022666931152344, Freq Loss: 0.0216064453125
Learning Rate: 0.0004800000000000001
Train Accuracy: 0.361682265996933
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 7038803968
Current train time: 44.26400375366211 s
Throughout: 1129.5860238549553 samples per second
Val Loss: 1.4140625
Val Accuracy: 0.49320000410079956


Epoch: 6
Train Loss: 1.7163701057434082
CE Loss: 1.6940312385559082, Freq Loss: 0.0223388671875
Learning Rate: 0.0004960000000000001
Train Accuracy: 0.388952374458313
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 7038803968
Current train time: 44.06972479248047 s
Throughout: 1134.5657417976752 samples per second
Val Loss: 1.3125
Val Accuracy: 0.5007999539375305


Epoch: 7
Train Loss: 1.7100353240966797
CE Loss: 1.6885509490966797, Freq Loss: 0.021484375
Learning Rate: 0.0005120000000000001
Train Accuracy: 0.42148837447166443
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 7038803968
Current train time: 44.114796051025394 s
Throughout: 1133.4065772891138 samples per second
Val Loss: 1.2734375
Val Accuracy: 0.5416999459266663


Epoch: 8
Train Loss: 1.5519883632659912
CE Loss: 1.5305039882659912, Freq Loss: 0.021484375
Learning Rate: 0.000528
Train Accuracy: 0.44404399394989014
Peak Mem Reserved: 10240393216
Peak Mem Allocated: 7038803968
Current train time: 44.62808120727539 s
Throughout: 1120.370821406699 samples per second
Val Loss: 1.3203125
Val Accuracy: 0.5544000267982483



SyntaxError: invalid syntax (1577900566.py, line 2)

In [ ]:
controller.meta

: 

In [ ]:
Epoch: 0
Train Loss: 2.0204086303710938
CE Loss: 2.0204086303710938
Learning Rate: 3.0000000000000004e-05
Train Accuracy: 0.19953462481498718
Peak Mem Reserved: 27648851968
Peak Mem Allocated: 26464001024
Current train time: 34.63649951171875 s
Throughout: 1443.5638908338076 samples per second
Val Loss: 1.78125
Val Accuracy: 0.31870001554489136


Epoch: 1
Train Loss: 1.8824348449707031
CE Loss: 1.8824348449707031
Learning Rate: 0.00012
Train Accuracy: 0.2547570466995239
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.778107513427734 s
Throughout: 1624.5313321550464 samples per second
Val Loss: 1.6953125
Val Accuracy: 0.3573000133037567


Epoch: 2
Train Loss: 1.8131141662597656
CE Loss: 1.8131141662597656
Learning Rate: 0.00021
Train Accuracy: 0.31078386306762695
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.613320495605468 s
Throughout: 1633.2759462397255 samples per second
/home/hice1/yyu496/.conda/envs/lib/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
Val Loss: 1.6015625
Val Accuracy: 0.3857000172138214


Epoch: 3
Train Loss: 1.6698455810546875
CE Loss: 1.6698455810546875
Learning Rate: 0.00030000000000000003
Train Accuracy: 0.34447360038757324
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.608947814941406 s
Throughout: 1633.5092699786653 samples per second
Val Loss: 1.390625
Val Accuracy: 0.48270002007484436


Epoch: 4
Train Loss: 1.589508056640625
CE Loss: 1.589508056640625
Learning Rate: 0.00029999723814541575
Train Accuracy: 0.3940315246582031
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.61799234008789 s
Throughout: 1633.0267329296898 samples per second
Val Loss: 1.3046875
Val Accuracy: 0.5138000249862671


Epoch: 5
Train Loss: 1.5945444107055664
CE Loss: 1.5945444107055664
Learning Rate: 0.0002999889526868746
Train Accuracy: 0.41792812943458557
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.601851348876952 s
Throughout: 1633.888075266235 samples per second
Val Loss: 1.3515625
Val Accuracy: 0.5182999968528748


Epoch: 6
Train Loss: 1.496464729309082
CE Loss: 1.496464729309082
Learning Rate: 0.0002999751439400074
Train Accuracy: 0.4448806643486023
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 31.114970428466798 s
Throughout: 1606.9435166248934 samples per second
Val Loss: 1.265625
Val Accuracy: 0.5572999119758606


Epoch: 7
Train Loss: 1.44968581199646
CE Loss: 1.44968581199646
Learning Rate: 0.000299955812430852
Train Accuracy: 0.46641361713409424
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.606640563964845 s
Throughout: 1633.6324104406349 samples per second
Val Loss: 1.1875
Val Accuracy: 0.566800057888031


Epoch: 8
Train Loss: 1.4654741287231445
CE Loss: 1.4654741287231445
Learning Rate: 0.00029993095889583346
Train Accuracy: 0.4787924587726593
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.597997955322267 s
Throughout: 1634.0938408129712 samples per second
Val Loss: 1.21875
Val Accuracy: 0.5699999928474426


Epoch: 9
Train Loss: 1.3801345825195312
CE Loss: 1.3801345825195312
Learning Rate: 0.000299900584281736
Train Accuracy: 0.4986399710178375
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.598928466796874 s
Throughout: 1634.0441481228786 samples per second
Val Loss: 1.140625
Val Accuracy: 0.5840999484062195


Epoch: 10
Train Loss: 1.3266618251800537
CE Loss: 1.3266618251800537
Learning Rate: 0.00029986468974566667
Train Accuracy: 0.5131911635398865
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.615754669189453 s
Throughout: 1633.1460890075045 samples per second
Val Loss: 1.203125
Val Accuracy: 0.5795000195503235


Epoch: 11
Train Loss: 1.4000189304351807
CE Loss: 1.4000189304351807
Learning Rate: 0.00029982327665501157
Train Accuracy: 0.528613805770874
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.606928527832032 s
Throughout: 1633.617040485886 samples per second
Val Loss: 1.109375
Val Accuracy: 0.6019999980926514


Epoch: 12
Train Loss: 1.2096363306045532
CE Loss: 1.2096363306045532
Learning Rate: 0.00029977634658738355
Train Accuracy: 0.5367401242256165
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.592514007568358 s
Throughout: 1634.3867649330937 samples per second
Val Loss: 1.1875
Val Accuracy: 0.6022999882698059


Epoch: 13
Train Loss: 1.2830979824066162
CE Loss: 1.2830979824066162
Learning Rate: 0.0002997239013305623
Train Accuracy: 0.5472036600112915
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.602636810302734 s
Throughout: 1633.8461391394521 samples per second
Val Loss: 0.99609375
Val Accuracy: 0.652999997138977


Epoch: 14
Train Loss: 1.1729369163513184
CE Loss: 1.1729369163513184
Learning Rate: 0.000299665942882426
Train Accuracy: 0.5562751889228821
Peak Mem Reserved: 28089253888
Peak Mem Allocated: 26627784192
Current train time: 30.595804107666016 s
Throughout: 1634.2110122045171 samples per second
Val Loss: 1.03125
Val Accuracy: 0.6360999941825867

: 

In [ ]:
Epoch: 0
Train Loss: 1.9045333862304688
CE Loss: 1.8829269409179688, Freq Loss: 0.0216064453125
Learning Rate: 0.0004
Train Accuracy: 0.2390061914920807
Peak Mem Reserved: 9544138752
Peak Mem Allocated: 6891101696
Current train time: 42.02726989746094 s
Throughout: 1189.7037357408917 samples per second
Val Loss: 1.734375
Val Accuracy: 0.36089998483657837


Epoch: 1
Train Loss: 1.6597986221313477
CE Loss: 1.6379480361938477, Freq Loss: 0.0218505859375
Learning Rate: 0.00041600000000000003
Train Accuracy: 0.3497304320335388
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 29.644499908447266 s
Throughout: 1686.6535159782673 samples per second
Val Loss: 1.4375
Val Accuracy: 0.47850000858306885


Epoch: 2
Train Loss: 1.563159465789795
CE Loss: 1.540820598602295, Freq Loss: 0.0223388671875
Learning Rate: 0.0004320000000000001
Train Accuracy: 0.4164164066314697
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 30.015953643798827 s
Throughout: 1665.7808242027918 samples per second
Val Loss: 1.296875
Val Accuracy: 0.5196999907493591


Epoch: 3
Train Loss: 1.47509765625
CE Loss: 1.4534912109375, Freq Loss: 0.0216064453125
Learning Rate: 0.00044800000000000005
Train Accuracy: 0.4653458297252655
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 29.964562896728516 s
Throughout: 1668.6377229103155 samples per second
Val Loss: 1.3515625
Val Accuracy: 0.5493999719619751


Epoch: 4
Train Loss: 1.3040845394134521
CE Loss: 1.2834546566009521, Freq Loss: 0.0206298828125
Learning Rate: 0.0004640000000000001
Train Accuracy: 0.503715991973877
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 29.582514770507814 s
Throughout: 1690.1876121041382 samples per second
Val Loss: 1.125
Val Accuracy: 0.6157000064849854


Epoch: 5
Train Loss: 1.1938207149505615
CE Loss: 1.1712377071380615, Freq Loss: 0.0225830078125
Learning Rate: 0.0004800000000000001
Train Accuracy: 0.5403010249137878
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 29.67847998046875 s
Throughout: 1684.722399290824 samples per second
Val Loss: 1.0546875
Val Accuracy: 0.6433999538421631


Epoch: 6
Train Loss: 1.2692553997039795
CE Loss: 1.2474048137664795, Freq Loss: 0.0218505859375
Learning Rate: 0.0004960000000000001
Train Accuracy: 0.5698992013931274
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 29.732331970214844 s
Throughout: 1681.6709853128518 samples per second
Val Loss: 0.9921875
Val Accuracy: 0.6561999917030334


Epoch: 7
Train Loss: 1.1457955837249756
CE Loss: 1.1237008571624756, Freq Loss: 0.0220947265625
Learning Rate: 0.0005120000000000001
Train Accuracy: 0.5942196846008301
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 29.795576751708985 s
Throughout: 1678.101431519769 samples per second
Val Loss: 0.87890625
Val Accuracy: 0.7031000852584839


Epoch: 8
Train Loss: 1.0809234380722046
CE Loss: 1.0595611333847046, Freq Loss: 0.0213623046875
Learning Rate: 0.000528
Train Accuracy: 0.6160621643066406
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 29.909658966064452 s
Throughout: 1671.7007725407395 samples per second
Val Loss: 0.86328125
Val Accuracy: 0.7122999429702759


Epoch: 9
Train Loss: 1.0484752655029297
CE Loss: 1.0272350311279297, Freq Loss: 0.021240234375
Learning Rate: 0.000544
Train Accuracy: 0.6350563764572144
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 30.036337615966797 s
Throughout: 1664.6503524924046 samples per second
Val Loss: 0.8515625
Val Accuracy: 0.7229999899864197


Epoch: 10
Train Loss: 1.1214923858642578
CE Loss: 1.0984210968017578, Freq Loss: 0.0230712890625
Learning Rate: 0.00056
Train Accuracy: 0.6488553285598755
Peak Mem Reserved: 10194255872
Peak Mem Allocated: 7054884864
Current train time: 29.445481689453125 s
Throughout: 1698.0533898995159 samples per second
Val Loss: 0.78125
Val Accuracy: 0.7470000386238098


Epoch: 11
Train Loss: 0.9483517408370972
CE Loss: 0.9258908033370972, Freq Loss: 0.0224609375
Learning Rate: 0.0005759999999999999
Train Accuracy: 0.6640090942382812
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 29.650903198242187 s
Throughout: 1686.2892730688952 samples per second
Val Loss: 0.8125
Val Accuracy: 0.7394000291824341


Epoch: 12
Train Loss: 0.9587233662605286
CE Loss: 0.9379714131355286, Freq Loss: 0.020751953125
Learning Rate: 0.0005919999999999999
Train Accuracy: 0.671798050403595
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 29.906405731201172 s
Throughout: 1671.8826210478146 samples per second
Val Loss: 0.80078125
Val Accuracy: 0.7578000426292419


Epoch: 13
Train Loss: 0.9989405870437622
CE Loss: 0.9780665636062622, Freq Loss: 0.0208740234375
Learning Rate: 0.0006079999999999998
Train Accuracy: 0.6846007108688354
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 29.945278594970702 s
Throughout: 1669.7123001018758 samples per second
Val Loss: 0.65234375
Val Accuracy: 0.7644000053405762


Epoch: 14
Train Loss: 0.9170495271682739
CE Loss: 0.8961755037307739, Freq Loss: 0.0208740234375
Learning Rate: 0.0006239999999999999
Train Accuracy: 0.6935915350914001
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 29.657180267333985 s
Throughout: 1685.9323627294632 samples per second
Val Loss: 0.68359375
Val Accuracy: 0.776199996471405


Epoch: 15
Train Loss: 0.9113831520080566
CE Loss: 0.8897767066955566, Freq Loss: 0.0216064453125
Learning Rate: 0.0006399999999999998
Train Accuracy: 0.6990640163421631
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 29.358371978759767 s
Throughout: 1703.0917121758 samples per second
Val Loss: 0.6328125
Val Accuracy: 0.7851999998092651


Epoch: 16
Train Loss: 0.7893111109733582
CE Loss: 0.7670943140983582, Freq Loss: 0.022216796875
Learning Rate: 0.0006559999999999998
Train Accuracy: 0.7096731066703796
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 30.126253051757814 s
Throughout: 1659.6820027402175 samples per second
Val Loss: 0.69921875
Val Accuracy: 0.7835999727249146


Epoch: 17
Train Loss: 0.8384559154510498
CE Loss: 0.8168494701385498, Freq Loss: 0.0216064453125
Learning Rate: 0.0006719999999999997
Train Accuracy: 0.7116010189056396
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 30.51877737426758 s
Throughout: 1638.3356183252067 samples per second
Val Loss: 0.67578125
Val Accuracy: 0.7857000827789307


Epoch: 18
Train Loss: 0.8948007822036743
CE Loss: 0.8728281259536743, Freq Loss: 0.02197265625
Learning Rate: 0.0006879999999999997
Train Accuracy: 0.717888593673706
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 29.180453094482424 s
Throughout: 1713.47579278864 samples per second
Val Loss: 0.56640625
Val Accuracy: 0.7947999835014343


Epoch: 19
Train Loss: 0.8306549787521362
CE Loss: 0.8099030256271362, Freq Loss: 0.020751953125
Learning Rate: 0.0007039999999999998
Train Accuracy: 0.7262073755264282
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 29.970761840820312 s
Throughout: 1668.2925934802156 samples per second
Val Loss: 0.59375
Val Accuracy: 0.8078001141548157


Epoch: 20
Train Loss: 0.8981999754905701
CE Loss: 0.8763493895530701, Freq Loss: 0.0218505859375
Learning Rate: 0.0007199999999999997
Train Accuracy: 0.7332481145858765
Peak Mem Reserved: 10236198912
Peak Mem Allocated: 7054884864
Current train time: 29.82004916381836 s
Throughout: 1676.7242644477808 samples per second
Val Loss: 0.5703125
Val Accuracy: 0.801800012588501

: 

In [ ]:
for n, m in controller.traced_model.named_modules():
    if isinstance(m, (DOConv2d, DOLinear)):
        print(n)

: 

In [ ]:
controller.init_acc_var

: 

In [ ]:
Epoch: 0
Train Loss: 2.0159902572631836
CE Loss: 2.0147390365600586, Freq Loss: 0.001251220703125
Learning Rate: 0.0004
Train Accuracy: 0.2260379195213318
Peak Mem Reserved: 26833059840
Peak Mem Allocated: 26671563776
Current train time: 42.80251544189453 s
Throughout: 1168.1556442138601 samples per second
Val Loss: 1.6875
Val Accuracy: 0.3934999704360962


Epoch: 1
Train Loss: 1.8517332077026367
CE Loss: 1.851038932800293, Freq Loss: 0.00069427490234375
Learning Rate: 0.00041600000000000003
Train Accuracy: 0.34172680974006653
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 30.560564727783202 s
Throughout: 1636.09542053207 samples per second
Val Loss: 1.5
Val Accuracy: 0.45990002155303955


Epoch: 2
Train Loss: 1.6418410539627075
CE Loss: 1.64130699634552, Freq Loss: 0.0005340576171875
Learning Rate: 0.0004320000000000001
Train Accuracy: 0.4238045811653137
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 29.61539227294922 s
Throughout: 1688.3112517699162 samples per second
Val Loss: 1.328125
Val Accuracy: 0.5654000043869019


Epoch: 3
Train Loss: 1.5153136253356934
CE Loss: 1.51485013961792, Freq Loss: 0.0004634857177734375
Learning Rate: 0.00044800000000000005
Train Accuracy: 0.4903367757797241
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 29.604291748046876 s
Throughout: 1688.9443066409017 samples per second
Val Loss: 1.15625
Val Accuracy: 0.614799976348877


Epoch: 4
Train Loss: 1.45391845703125
CE Loss: 1.4534492492675781, Freq Loss: 0.000469207763671875
Learning Rate: 0.0004640000000000001
Train Accuracy: 0.5410183072090149
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 29.813560211181642 s
Throughout: 1677.0892052418278 samples per second
Val Loss: 1.0546875
Val Accuracy: 0.6564000248908997


Epoch: 5
Train Loss: 1.4149445295333862
CE Loss: 1.414528727531433, Freq Loss: 0.000415802001953125
Learning Rate: 0.0004800000000000001
Train Accuracy: 0.5802582502365112
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 29.246568756103514 s
Throughout: 1709.602258540685 samples per second
Val Loss: 0.96484375
Val Accuracy: 0.6853000521659851


Epoch: 6
Train Loss: 1.3319803476333618
CE Loss: 1.3314844369888306, Freq Loss: 0.00049591064453125
Learning Rate: 0.0004960000000000001
Train Accuracy: 0.6139059662818909
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 29.415003631591798 s
Throughout: 1699.8128107079292 samples per second
Val Loss: 0.89453125
Val Accuracy: 0.7192999720573425


Epoch: 7
Train Loss: 1.238694190979004
CE Loss: 1.2382097244262695, Freq Loss: 0.000484466552734375
Learning Rate: 0.0005120000000000001
Train Accuracy: 0.6393776535987854
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 29.029764465332033 s
Throughout: 1722.370157694909 samples per second
Val Loss: 0.88671875
Val Accuracy: 0.7364000082015991


Epoch: 8
Train Loss: 1.281731128692627
CE Loss: 1.2812199592590332, Freq Loss: 0.00051116943359375
Learning Rate: 0.000528
Train Accuracy: 0.6619654893875122
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 30.283424713134767 s
Throughout: 1651.06821548864 samples per second
Val Loss: 0.76171875
Val Accuracy: 0.7518000602722168


Epoch: 9
Train Loss: 1.2240803241729736
CE Loss: 1.2235958576202393, Freq Loss: 0.000484466552734375
Learning Rate: 0.000544
Train Accuracy: 0.6812035441398621
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 29.512437561035156 s
Throughout: 1694.2009583787913 samples per second
Val Loss: 0.69921875
Val Accuracy: 0.7717999815940857


Epoch: 10
Train Loss: 1.166072130203247
CE Loss: 1.1657040119171143, Freq Loss: 0.0003681182861328125
Learning Rate: 0.00056
Train Accuracy: 0.6928081512451172
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 30.05364080810547 s
Throughout: 1663.6919406621441 samples per second
Val Loss: 0.7265625
Val Accuracy: 0.7827000617980957


Epoch: 11
Train Loss: 1.183124303817749
CE Loss: 1.1826531887054443, Freq Loss: 0.0004711151123046875
Learning Rate: 0.0005759999999999999
Train Accuracy: 0.7089468836784363
Peak Mem Reserved: 27015512064
Peak Mem Allocated: 26824751616
Current train time: 30.691831298828124 s
Throughout: 1629.0979678983542 samples per second
Val Loss: 0.66015625
Val Accuracy: 0.8001999855041504

: 

In [ ]:
Epoch: 0
Train Loss: 1.972098469734192
CE Loss: 1.947684407234192, Freq Loss: 0.0244140625
Learning Rate: 0.0004
Train Accuracy: 0.2394513040781021
Peak Mem Reserved: 26042433536
Peak Mem Allocated: 25853588480
Current train time: 39.60026376342773 s
Throughout: 1262.6178527168497 samples per second
Val Loss: 1.75
Val Accuracy: 0.326200008392334


Epoch: 1
Train Loss: 1.7997024059295654
CE Loss: 1.7756545543670654, Freq Loss: 0.0240478515625
Learning Rate: 0.00041600000000000003
Train Accuracy: 0.3568476140499115
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 29.502670318603517 s
Throughout: 1694.7618456242406 samples per second
Val Loss: 1.46875
Val Accuracy: 0.4838999807834625


Epoch: 2
Train Loss: 1.6860631704330444
CE Loss: 1.6638463735580444, Freq Loss: 0.022216796875
Learning Rate: 0.0004320000000000001
Train Accuracy: 0.4278605282306671
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 28.261893188476563 s
Throughout: 1769.166689101595 samples per second
Val Loss: 1.3671875
Val Accuracy: 0.5310999751091003


Epoch: 3
Train Loss: 1.5963082313537598
CE Loss: 1.5745797157287598, Freq Loss: 0.021728515625
Learning Rate: 0.00044800000000000005
Train Accuracy: 0.4847506284713745
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 29.090908630371093 s
Throughout: 1718.7500272095208 samples per second
Val Loss: 1.25
Val Accuracy: 0.6098999977111816


Epoch: 4
Train Loss: 1.4873754978179932
CE Loss: 1.4660131931304932, Freq Loss: 0.0213623046875
Learning Rate: 0.0004640000000000001
Train Accuracy: 0.5255968570709229
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 29.415791107177736 s
Throughout: 1699.7673058604064 samples per second
Val Loss: 1.1796875
Val Accuracy: 0.6085999608039856


Epoch: 5
Train Loss: 1.44301438331604
CE Loss: 1.42067551612854, Freq Loss: 0.0223388671875
Learning Rate: 0.0004800000000000001
Train Accuracy: 0.5670493841171265
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 28.817663848876954 s
Throughout: 1735.0469580811819 samples per second
Val Loss: 1.0390625
Val Accuracy: 0.6661999225616455


Epoch: 6
Train Loss: 1.4052331447601318
CE Loss: 1.3835046291351318, Freq Loss: 0.021728515625
Learning Rate: 0.0004960000000000001
Train Accuracy: 0.5971462726593018
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 28.694834228515624 s
Throughout: 1742.473910175521 samples per second
Val Loss: 0.9765625
Val Accuracy: 0.6917000412940979


Epoch: 7
Train Loss: 1.3891098499298096
CE Loss: 1.3667709827423096, Freq Loss: 0.0223388671875
Learning Rate: 0.0005120000000000001
Train Accuracy: 0.6229101419448853
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 28.827572143554686 s
Throughout: 1734.4506069054823 samples per second
Val Loss: 0.91015625
Val Accuracy: 0.6922000050544739


Epoch: 8
Train Loss: 1.3049952983856201
CE Loss: 1.2841212749481201, Freq Loss: 0.0208740234375
Learning Rate: 0.000528
Train Accuracy: 0.6431524157524109
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 28.97141421508789 s
Throughout: 1725.8391195125273 samples per second
Val Loss: 0.8203125
Val Accuracy: 0.7429000735282898


Epoch: 9
Train Loss: 1.2998377084732056
CE Loss: 1.2771326303482056, Freq Loss: 0.022705078125
Learning Rate: 0.000544
Train Accuracy: 0.6578047275543213
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 29.156133239746094 s
Throughout: 1714.9050454961985 samples per second
Val Loss: 0.83203125
Val Accuracy: 0.73580002784729


Epoch: 10
Train Loss: 1.2484318017959595
CE Loss: 1.2259708642959595, Freq Loss: 0.0224609375
Learning Rate: 0.00056
Train Accuracy: 0.6726032495498657
Peak Mem Reserved: 26185039872
Peak Mem Allocated: 26006776320
Current train time: 29.73736404418945 s
Throughout: 1681.3864176293655 samples per second
Val Loss: 0.75
Val Accuracy: 0.7583000063896179

: 

In [ ]:
Epoch: 0
Train Loss: 1.8929107189178467
CE Loss: 1.8708159923553467, Freq Loss: 0.0220947265625
Learning Rate: 0.0004
Train Accuracy: 0.25595182180404663
Peak Mem Reserved: 26000490496
Peak Mem Allocated: 25810187264
Current train time: 40.51330209350586 s
Throughout: 1234.1625445538498 samples per second
Val Loss: 1.640625
Val Accuracy: 0.4020000100135803


Epoch: 1
Train Loss: 1.8045774698257446
CE Loss: 1.7823606729507446, Freq Loss: 0.022216796875
Learning Rate: 0.00041600000000000003
Train Accuracy: 0.3834226429462433
Peak Mem Reserved: 26140999680
Peak Mem Allocated: 25963375104
Current train time: 28.617461181640625 s
Throughout: 1747.1850379263283 samples per second
Val Loss: 1.4296875
Val Accuracy: 0.49400001764297485


Epoch: 2
Train Loss: 1.6234170198440552
CE Loss: 1.6009560823440552, Freq Loss: 0.0224609375
Learning Rate: 0.0004320000000000001
Train Accuracy: 0.45989999175071716
Peak Mem Reserved: 26140999680
Peak Mem Allocated: 25963375104
Current train time: 28.550537170410156 s
Throughout: 1751.2805346380705 samples per second
Val Loss: 1.2109375
Val Accuracy: 0.5890999436378479


Epoch: 3
Train Loss: 1.5424914360046387
CE Loss: 1.5210070610046387, Freq Loss: 0.021484375
Learning Rate: 0.00044800000000000005
Train Accuracy: 0.5186222791671753
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.0019514465332 s
Throughout: 1724.021919427661 samples per second
Val Loss: 1.1640625
Val Accuracy: 0.6300000548362732


Epoch: 4
Train Loss: 1.4019957780838013
CE Loss: 1.3801451921463013, Freq Loss: 0.0218505859375
Learning Rate: 0.0004640000000000001
Train Accuracy: 0.5643259286880493
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.172721282958985 s
Throughout: 1713.9299249812223 samples per second
Val Loss: 1.0390625
Val Accuracy: 0.6618999242782593


Epoch: 5
Train Loss: 1.4317731857299805
CE Loss: 1.4106550216674805, Freq Loss: 0.0211181640625
Learning Rate: 0.0004800000000000001
Train Accuracy: 0.591052770614624
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.450718688964844 s
Throughout: 1697.7514378531941 samples per second
Val Loss: 0.9921875
Val Accuracy: 0.6907999515533447


Epoch: 6
Train Loss: 1.3509769439697266
CE Loss: 1.3298587799072266, Freq Loss: 0.0211181640625
Learning Rate: 0.0004960000000000001
Train Accuracy: 0.6162897348403931
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 28.403817779541015 s
Throughout: 1760.3267415697371 samples per second
Val Loss: 0.92578125
Val Accuracy: 0.7064999938011169


Epoch: 7
Train Loss: 1.2641372680664062
CE Loss: 1.2414321899414062, Freq Loss: 0.022705078125
Learning Rate: 0.0005120000000000001
Train Accuracy: 0.6378638744354248
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.14587338256836 s
Throughout: 1715.5087220650635 samples per second
Val Loss: 0.83984375
Val Accuracy: 0.7432000041007996


Epoch: 8
Train Loss: 1.2883273363113403
CE Loss: 1.2672091722488403, Freq Loss: 0.0211181640625
Learning Rate: 0.000528
Train Accuracy: 0.6589770317077637
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.544981597900392 s
Throughout: 1692.3347822817138 samples per second
Val Loss: 0.84375
Val Accuracy: 0.7507999539375305


Epoch: 9
Train Loss: 1.2321743965148926
CE Loss: 1.2104458808898926, Freq Loss: 0.021728515625
Learning Rate: 0.000544
Train Accuracy: 0.6728095412254333
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.4812873840332 s
Throughout: 1695.991065406443 samples per second
Val Loss: 0.80078125
Val Accuracy: 0.7387999892234802


Epoch: 10
Train Loss: 1.0903221368789673
CE Loss: 1.0682274103164673, Freq Loss: 0.0220947265625
Learning Rate: 0.00056
Train Accuracy: 0.6910027265548706
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.68025408935547 s
Throughout: 1684.6216966158659 samples per second
Val Loss: 0.7578125
Val Accuracy: 0.7802000045776367


Epoch: 11
Train Loss: 1.209557056427002
CE Loss: 1.188560962677002, Freq Loss: 0.02099609375
Learning Rate: 0.0005759999999999999
Train Accuracy: 0.6994066834449768
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 28.971668426513673 s
Throughout: 1725.8239761656966 samples per second
Val Loss: 0.75390625
Val Accuracy: 0.7738000154495239


Epoch: 12
Train Loss: 1.156912088394165
CE Loss: 1.135061502456665, Freq Loss: 0.0218505859375
Learning Rate: 0.0005919999999999999
Train Accuracy: 0.7126305103302002
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.605869201660155 s
Throughout: 1688.8543166702987 samples per second
Val Loss: 0.75
Val Accuracy: 0.7758999466896057


Epoch: 13
Train Loss: 1.1251931190490723
CE Loss: 1.1040749549865723, Freq Loss: 0.0211181640625
Learning Rate: 0.0006079999999999998
Train Accuracy: 0.7202237248420715
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 30.14190576171875 s
Throughout: 1658.8201288686168 samples per second
Val Loss: 0.671875
Val Accuracy: 0.7989000082015991


Epoch: 14
Train Loss: 1.1366655826568604
CE Loss: 1.1149370670318604, Freq Loss: 0.021728515625
Learning Rate: 0.0006239999999999999
Train Accuracy: 0.7332989573478699
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.04398254394531 s
Throughout: 1721.5269952854076 samples per second
Val Loss: 0.6328125
Val Accuracy: 0.8105000257492065


Epoch: 15
Train Loss: 1.1325446367263794
CE Loss: 1.1110602617263794, Freq Loss: 0.021484375
Learning Rate: 0.0006399999999999998
Train Accuracy: 0.7374083399772644
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.372983154296875 s
Throughout: 1702.2445332620453 samples per second
Val Loss: 0.6328125
Val Accuracy: 0.814300000667572


Epoch: 16
Train Loss: 1.0633924007415771
CE Loss: 1.0415418148040771, Freq Loss: 0.0218505859375
Learning Rate: 0.0006559999999999998
Train Accuracy: 0.7451866865158081
Peak Mem Reserved: 26164068352
Peak Mem Allocated: 25963375104
Current train time: 29.934833374023437 s
Throughout: 1670.294916135679 samples per second
Val Loss: 0.578125
Val Accuracy: 0.8251000642776489

: 

In [ ]:
Epoch: 0
Train Loss: 2.171717643737793
CE Loss: 2.145960807800293, Freq Loss: 0.0257568359375
Learning Rate: 0.0004
Train Accuracy: 0.16836054623126984
Peak Mem Reserved: 26140999680
Peak Mem Allocated: 25960837120
Current train time: 38.90633868408203 s
Throughout: 1285.1376328674376 samples per second
Val Loss: 1.90625
Val Accuracy: 0.27300000190734863


Epoch: 1
Train Loss: 2.0348706245422363
CE Loss: 2.0111889839172363, Freq Loss: 0.023681640625
Learning Rate: 0.00041600000000000003
Train Accuracy: 0.2379419505596161
Peak Mem Reserved: 26308771840
Peak Mem Allocated: 26124620288
Current train time: 29.485845733642577 s
Throughout: 1695.728874513893 samples per second
Val Loss: 1.828125
Val Accuracy: 0.32580000162124634


Epoch: 2
Train Loss: 2.063933849334717
CE Loss: 2.040740489959717, Freq Loss: 0.023193359375
Learning Rate: 0.0004320000000000001
Train Accuracy: 0.2757069170475006
Peak Mem Reserved: 26308771840
Peak Mem Allocated: 26124620288
Current train time: 29.224626525878907 s
Throughout: 1710.8858501826924 samples per second
Val Loss: 1.6953125
Val Accuracy: 0.3812999725341797


Epoch: 3
Train Loss: 1.9659230709075928
CE Loss: 1.9426076412200928, Freq Loss: 0.0233154296875
Learning Rate: 0.00044800000000000005
Train Accuracy: 0.29685139656066895
Peak Mem Reserved: 26308771840
Peak Mem Allocated: 26124620288
Current train time: 28.638725494384765 s
Throughout: 1745.8877494322703 samples per second
Val Loss: 1.6875
Val Accuracy: 0.3930000066757202


Epoch: 4
Train Loss: 1.9245011806488037
CE Loss: 1.9013078212738037, Freq Loss: 0.023193359375
Learning Rate: 0.0004640000000000001
Train Accuracy: 0.3301340937614441
Peak Mem Reserved: 26308771840
Peak Mem Allocated: 26124620288
Current train time: 31.603548645019533 s
Throughout: 1582.100812842725 samples per second
Val Loss: 1.5078125
Val Accuracy: 0.45750001072883606

: 

In [ ]:
x, y = next(iter(check_loder))
x, y = x.to('cuda'), y.to('cuda')

: 

In [ ]:
set_seed(42)
# torch.set_rng_state(cached_rng_state['CPU'])
# torch.cuda.set_rng_state(cached_rng_state['GPU'])
num_classes = 10
# model = models.resnet18(weights=None)
# model.fc = nn.Linear(model.fc.in_features, num_classes)
model = models.efficientnet_b0(weights=None)
# model = models.efficientnet_v2_s(weights=None)
# model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
# model.apply(lambda m: apply_trunc_init_for_act(m, std=0.02))
# model.apply(init_fan_out)
# model.apply(lambda m: init_orthogonal_conv(m, gain=math.sqrt(2)))  # ReLU-friendly
# model.apply(lambda m: init_orthogonal_conv(m, gain=1.0))           # smoother spectrum
# model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=10)

disable_act_inplace(model)
# model.load_state_dict(baseline)
# init(model)
# for n, m in model.named_modules():
#     if isinstance(m, (nn.Conv1d, nn.Conv2d)):
#             cahced_init(m.weight)

# for n, m in model.named_modules():
#     init_orthogonal_conv(n, m, gain=1.0)

# for m in model.modules():
#     if isinstance(m, nn.Conv2d) and m.kernel_size == (1, 1):
#         spectral_norm(m)

model = model.to('cuda')


criterion = nn.CrossEntropyLoss()

: 

In [ ]:
model.load_state_dict(torch.load("/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192_acc_89"))

: 

In [ ]:
activation_cache.clear()
torch.cuda.empty_cache()

: 

In [ ]:
model.eval()
activation_cache = {}
low_frequency_energy_ratio_grad = {}
low_frequency_energy_ratio_act = {}


def forward_hook(name):
    def hook(module, input, output):
        activation_cache[name] = output.detach().cpu()
    return hook

@torch.no_grad()
def compute_lfer_act(x, k=4):
    if x.ndim == 4:
        H, W = x.shape[-2:]
        k = min(k, H, W)
        x_lp = F.interpolate(F.avg_pool2d(x, kernel_size=k, stride=k, padding=0), size=x.shape[-2:], mode='nearest')
        return (x_lp.square().sum() / (x.square().sum() + 1e-7)).item()
    else:
        feature_dim = x.shape[-1]
        k = min(k, feature_dim)
        if x.ndim != 2:
            pooled = F.avg_pool1d(x, kernel_size=k, stride=k, padding=0)
            x_lp = F.interpolate(pooled, size=feature_dim, mode='nearest')
        else:
            x_lp = x.mean(dim=-1, keepdim=True).expand_as(x)
        return (x_lp.square().sum() / (x.square().sum() + 1e-7)).item().detach().cpu()


for name, module in model.named_modules():
    if isinstance(module, (nn.Conv1d, nn.Conv2d)):
        module.register_forward_hook(forward_hook(name))

with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        logits = model(x)
        loss = criterion(logits, y)



: 

In [ ]:
activation_cache.keys()

: 

In [ ]:
def radial_spectrum_2d_per_sample(x, num_bins=64, eps=1e-8, out_dtype=None,
                                  *, vit_grid=None, num_prefix_tokens=None, max_prefix_tokens=16,
                                  log_power=True):
    if out_dtype is None:
        out_dtype = x.dtype

    # --- your ViT token -> [B,C,H,W] conversion stays the same ---
    if x.dim() == 3:
        import math
        B, N, D = x.shape
        if vit_grid is not None:
            H, W = vit_grid
            needed = H * W
            if num_prefix_tokens is None:
                num_prefix_tokens = N - needed
            if N - num_prefix_tokens != needed:
                raise ValueError("grid mismatch")
        else:
            if num_prefix_tokens is None:
                found = None
                for k in range(0, max_prefix_tokens + 1):
                    m = N - k
                    if m <= 0: break
                    s = int(math.isqrt(m))
                    if s * s == m:
                        found = k
                        H = W = s
                        break
                if found is None:
                    raise ValueError("cannot infer grid")
                num_prefix_tokens = found
            else:
                m = N - num_prefix_tokens
                s = int(math.isqrt(m))
                if s * s != m:
                    raise ValueError("not square")
                H = W = s

        if num_prefix_tokens > 0:
            x = x[:, num_prefix_tokens:, :].contiguous()
        x = x.view(B, H, W, D).permute(0, 3, 1, 2).contiguous()

    elif x.dim() != 4:
        raise ValueError(f"Expected dim 3 or 4, got {x.dim()}")

    # x: [B,C,H,W]
    x = x.mean(dim=1)  # [B,H,W]

    x_fft = x.to(torch.float32)
    F = torch.fft.fft2(x_fft)
    F = torch.fft.fftshift(F, dim=(-2, -1))
    P = (F.real * F.real + F.imag * F.imag)  # [B,H,W]
    if log_power:
        P = torch.log1p(P)

    B, H, W = P.shape
    yy, xx = torch.meshgrid(torch.arange(H, device=P.device),
                            torch.arange(W, device=P.device), indexing="ij")
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    r = torch.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r = (r / (r.max() + 1e-12) * (num_bins - 1)).to(torch.long)  # [H,W]
    r_flat = r.flatten()  # [HW]

    vals = P.reshape(B, -1)  # [B,HW]

    # per-sample scatter_add
    spec = torch.zeros((B, num_bins), device=P.device, dtype=torch.float32)
    spec.scatter_add_(1, r_flat[None, :].expand(B, -1), vals)

    spec = spec / (spec.sum(dim=1, keepdim=True) + eps)
    return spec.to(out_dtype)


def vit_spectral_slope_loss(tokens, num_bins=64, alpha=2.0, num_prefix_tokens=1):
    spec = radial_spectrum_2d_per_sample(
        tokens, num_bins=num_bins,
        num_prefix_tokens=num_prefix_tokens,
        log_power=True
    )  # [B,K]

    K = spec.shape[1]
    k = torch.arange(K, device=spec.device, dtype=spec.dtype)
    target = 1.0 / (k + 1.0) ** alpha
    target = target / target.sum()

    # compare each sample spectrum to 1/f^alpha
    return torch.mean(torch.sum(torch.abs(spec - target[None, :]), dim=1))

: 

In [ ]:
import math
import torch
import torch.nn.functional as F


def _infer_vit_grid(N, num_prefix_tokens=None, max_prefix_tokens=16):
    if num_prefix_tokens is not None:
        M = N - num_prefix_tokens
        s = int(math.isqrt(M))
        if s * s != M:
            raise ValueError(f"N - num_prefix_tokens = {M} not square")
        return s, s, num_prefix_tokens

    # auto infer
    for k in range(0, max_prefix_tokens + 1):
        M = N - k
        if M <= 0:
            break
        s = int(math.isqrt(M))
        if s * s == M:
            return s, s, k
    raise ValueError(f"Cannot infer vit grid from N={N}")


def _radial_bins(H, W, num_bins, device):
    yy, xx = torch.meshgrid(
        torch.arange(H, device=device),
        torch.arange(W, device=device),
        indexing="ij"
    )
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    r = torch.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r = (r / (r.max() + 1e-12) * (num_bins - 1)).to(torch.long)  # [H,W]
    return r


def local_radial_spectrum_2d(
    x,                      # [B,1,H,W] or [B,H,W]
    window=8,
    num_bins=32,
    eps=1e-8,
):
    """
    Returns local spectra per window:
      spectra: [B, nwin, num_bins] each normalized to sum=1
    """
    if x.dim() == 4:
        x = x.squeeze(1)
    assert x.dim() == 3
    B, H, W = x.shape

    # crop to divisible by window
    Hc = (H // window) * window
    Wc = (W // window) * window
    x = x[:, :Hc, :Wc]

    # unfold into windows: [B, nH, nW, window, window]
    nH = Hc // window
    nW = Wc // window
    xw = x.view(B, nH, window, nW, window).permute(0, 1, 3, 2, 4).contiguous()
    xw = xw.view(B, nH * nW, window, window)  # [B, nwin, w, w]

    # FFT per window (fp32)
    X = xw.to(torch.float32)
    Fw = torch.fft.fft2(X)                      # complex64
    Fw = torch.fft.fftshift(Fw, dim=(-2, -1))
    Pw = (Fw.real * Fw.real + Fw.imag * Fw.imag)  # [B,nwin,w,w]

    # build radial bins for (window, window)
    r = _radial_bins(window, window, num_bins, device=Pw.device)  # [w,w]
    r_flat = r.flatten()  # [w*w]

    # scatter-add radial power
    spectra = torch.zeros((B, nH * nW, num_bins), device=Pw.device, dtype=torch.float32)
    vals = Pw.reshape(B, nH * nW, -1)  # [B,nwin,w*w]

    # vectorized scatter: flatten over B*nwin
    BN = B * (nH * nW)
    spectra2 = torch.zeros((BN, num_bins), device=Pw.device, dtype=torch.float32)
    idx = r_flat[None, :].expand(BN, -1)  # [BN,w*w]
    spectra2.scatter_add_(1, idx, vals.reshape(BN, -1))

    spectra = spectra2.view(B, nH * nW, num_bins)
    spectra = spectra / (spectra.sum(dim=-1, keepdim=True) + eps)

    return spectra  # [B,nwin,K]


def local_freq_kl_loss(
    vit_tokens,             # [B,N,D] tokens (output of some block or final tokens)
    images,                 # [B,3,H_img,W_img] input image tensor
    *,
    vit_grid=None,          # (H,W) patch grid override
    num_prefix_tokens=None, # if known
    max_prefix_tokens=16,
    window=7,
    num_bins=32,
    low_freq_bins=16,       # only match first few bins
    eps=1e-8,
    detach_target=True,
):
    """
    Local spectrum KL loss:
      KL( target || model )

    Returns scalar loss.
    """
    assert vit_tokens.dim() == 3
    B, N, D = vit_tokens.shape

    # infer token grid
    if vit_grid is not None:
        Ht, Wt = vit_grid
        needed = Ht * Wt
        if num_prefix_tokens is None:
            num_prefix_tokens = N - needed
        assert N - num_prefix_tokens == needed
    else:
        Ht, Wt, num_prefix_tokens = _infer_vit_grid(N, num_prefix_tokens, max_prefix_tokens)

    # patch tokens only
    x = vit_tokens[:, num_prefix_tokens:, :]  # [B, Ht*Wt, D]
    x = x.view(B, Ht, Wt, D).permute(0, 3, 1, 2).contiguous()  # [B,D,Ht,Wt]

    # make it 1-channel "activation image"
    x_map = x.mean(dim=1, keepdim=True)  # [B,1,Ht,Wt]

    # downsample input image to patch grid resolution
    img = images
    img_gray = img.mean(dim=1, keepdim=True)  # [B,1,H_img,W_img]
    img_ds = F.interpolate(img_gray, size=(Ht, Wt), mode="bilinear", align_corners=False)

    if detach_target:
        img_ds = img_ds.detach()

    # compute local spectra
    spec_model = local_radial_spectrum_2d(x_map, window=window, num_bins=num_bins, eps=eps)  # [B,nwin,K]
    spec_tgt   = local_radial_spectrum_2d(img_ds, window=window, num_bins=num_bins, eps=eps) # [B,nwin,K]

    # use only low-freq bins
    K = min(low_freq_bins, num_bins)
    pm = spec_model[..., :K].clamp_min(eps)
    pt = spec_tgt[..., :K].clamp_min(eps)

    # renormalize after truncation
    pm = pm / pm.sum(dim=-1, keepdim=True)
    pt = pt / pt.sum(dim=-1, keepdim=True)

    # KL(target || model)
    loss = (pt * (pt.log() - pm.log())).sum(dim=-1)  # [B,nwin]
    return loss.mean()


: 

In [ ]:
import math
import torch
import torch.nn.functional as F

import math
import torch
import torch.nn.functional as F


def radial_spectrum_2d(
    x,
    num_bins=128,
    eps=1e-8,
    out_dtype=None,
    *,
    vit_grid=None,
    num_prefix_tokens=None,
    max_prefix_tokens=16,
    use_cls_token=True,
    # ---- NEW ----
    num_theta_bins=360,
    remove_dc=True,
    power=True,
    theta_offset=0.0,
    return_marginal_r=False,
):
    """
    Returns:
      polar spectrum [num_bins, num_theta_bins] (distribution), differentiable
    """
    if out_dtype is None:
        out_dtype = x.dtype

    # ---------------------------
    # ViT tokens -> [B,C,H,W]
    # ---------------------------
    if x.dim() == 3:
        B, N, D = x.shape

        if vit_grid is not None:
            H, W = vit_grid
            needed = H * W
            if num_prefix_tokens is None:
                num_prefix_tokens = N - needed
            if N - num_prefix_tokens != needed:
                raise ValueError(
                    f"vit_grid={vit_grid} implies {needed} patch tokens, but got N={N} "
                    f"with num_prefix_tokens={num_prefix_tokens} => {N - num_prefix_tokens} tokens."
                )
        else:
            if num_prefix_tokens is None:
                found = None
                for k in range(0, max_prefix_tokens + 1):
                    m = N - k
                    if m <= 0:
                        break
                    s = int(math.isqrt(m))
                    if s * s == m:
                        found = k
                        H = W = s
                        break
                if found is None:
                    raise ValueError(
                        f"Cannot infer square token grid from N={N}. "
                        f"Pass vit_grid=(H,W) or num_prefix_tokens explicitly."
                    )
                num_prefix_tokens = found
            else:
                m = N - num_prefix_tokens
                s = int(math.isqrt(m))
                if s * s != m:
                    raise ValueError(
                        f"N - num_prefix_tokens = {m} is not a perfect square. "
                        f"Pass vit_grid=(H,W) instead."
                    )
                H = W = s

        if use_cls_token:
            cls = x[:, 0, :]  # [B,D]
            HW = H * W
            if D < HW:
                cls = F.pad(cls, (0, HW - D), value=0.0)
            elif D > HW:
                cls = cls[:, :HW]
            x = cls.view(B, 1, H, W).contiguous()
        else:
            tok = x[:, num_prefix_tokens:, :] if num_prefix_tokens > 0 else x
            tok = tok.contiguous()
            tok = tok.view(B, H, W, D).permute(0, 3, 1, 2).contiguous()
            x = tok

    elif x.dim() != 4:
        raise ValueError(f"Expected x dim 3 or 4, got {x.dim()}")

    # ---------------------------
    # x: [B,C,H,W]
    # ---------------------------
    x = x.mean(dim=1)  # [B,H,W]

    X = torch.fft.fft2(x.to(torch.float32), dim=(-2, -1))
    X = torch.fft.fftshift(X, dim=(-2, -1))
    mag = X.abs()
    P = mag * mag if power else mag  # [B,H,W]

    B, H, W = P.shape

    if remove_dc:
        P[:, H // 2, W // 2] = 0.0

    # (r, theta) bin maps (NOT dependent on x, safe to compute)
    yy, xx = torch.meshgrid(
        torch.arange(H, device=P.device),
        torch.arange(W, device=P.device),
        indexing="ij"
    )
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    dy = yy - cy
    dx = xx - cx

    # radius bins
    r = torch.sqrt(dx * dx + dy * dy)
    r = r / (r.max() + 1e-12)
    r_bin = torch.clamp((r * (num_bins - 1)).long(), 0, num_bins - 1)

    # theta bins
    theta = torch.atan2(dy, dx) + theta_offset
    theta = (theta + math.pi) / (2.0 * math.pi)   # [0,1) (wrapped)
    theta = theta - torch.floor(theta)
    t_bin = torch.clamp((theta * num_theta_bins).long(), 0, num_theta_bins - 1)

    # Flatten indexing: joint bin id in [0, num_bins*num_theta_bins)
    joint = (r_bin * num_theta_bins + t_bin).reshape(-1)  # [HW]
    K = num_bins * num_theta_bins

    # Differentiable accumulation:
    # hist[k] = sum_{b,pixel where joint==k} P[b,pixel]
    # Use index_add_ (supports autograd w.r.t. P)
    hist = torch.zeros(K, device=P.device, dtype=P.dtype)

    # flatten P -> [B*HW]
    P_flat = P.reshape(-1)  # differentiable
    joint_flat = joint.repeat(B)  # [B*HW], integer indices

    hist.index_add_(0, joint_flat, P_flat)

    polar = hist.view(num_bins, num_theta_bins).to(torch.float32)

    polar = polar / (polar.sum() + eps)

    if return_marginal_r:
        spectrum_r = polar.sum(dim=1)
        spectrum_r = spectrum_r / (spectrum_r.sum() + eps)
        return polar.to(out_dtype), spectrum_r.to(out_dtype)

    return polar.to(out_dtype)




def polar_kl_loss(polar_s, polar_t, eps=1e-8, T=1.0, reduction="sum"):
    """
    polar_s, polar_t: [R,Theta] distributions (NOT raw energies)
    returns scalar
    """
    ps = polar_s.to(torch.float32).clamp_min(eps)
    pt = polar_t.to(torch.float32).clamp_min(eps)

    # re-normalize (safe)
    ps = ps / (ps.sum() + eps)
    pt = pt / (pt.sum() + eps)

    log_ps = torch.log(ps)
    log_pt = torch.log(pt)

    # temperature smoothing along theta dimension (can also use flatten)
    psT = F.softmax(log_ps.view(-1) / T, dim=0)
    ptT = F.softmax(log_pt.view(-1) / T, dim=0)

    kl = (psT * (torch.log(psT.clamp_min(eps)) - torch.log(ptT.clamp_min(eps)))).sum()

    if reduction == "mean":
        kl = kl / ps.numel()

    return (T * T) * kl


def wasserstein_1d(p, q, eps=1e-8):
    """
    p,q: [K] distributions
    """
    p32 = p.to(torch.float32)
    q32 = q.to(torch.float32)
    p32 = p32 / (p32.sum() + eps)
    q32 = q32 / (q32.sum() + eps)
    cdf_p = torch.cumsum(p32, dim=0)
    cdf_q = torch.cumsum(q32, dim=0)
    return torch.sum(torch.abs(cdf_p - cdf_q))


def polar_wasserstein_rtheta(polar_s, polar_t, eps=1e-8):
    """
    polar_s, polar_t: [R,Theta] distributions
    returns scalar
    """
    ps = polar_s.to(torch.float32).clamp_min(eps)
    pt = polar_t.to(torch.float32).clamp_min(eps)

    ps = ps / (ps.sum() + eps)
    pt = pt / (pt.sum() + eps)

    # 1) Wasserstein on radial marginal
    ps_r = ps.sum(dim=1)  # [R]
    pt_r = pt.sum(dim=1)
    wr = wasserstein_1d(ps_r, pt_r, eps=eps)

    # 2) Wasserstein on angular bins per radius
    # weight by average mass in that radius
    weights = 0.5 * (ps_r + pt_r)  # [R]
    wa = 0.0
    R = ps.shape[0]
    for r in range(R):
        wa = wa + weights[r] * wasserstein_1d(ps[r], pt[r], eps=eps)

    return wr + wa


: 

In [ ]:
# polar_spectrum.py
import math
from dataclasses import dataclass
from typing import Dict, Tuple, Optional

import torch
import torch.nn.functional as F


# -------------------------
# Bin-cache (critical for speed)
# -------------------------
@dataclass
class _BinKey:
    H: int
    W: int
    num_r: int
    num_t: int
    theta_offset: float

class PolarBinCache:
    """
    Caches joint bin indices for (H,W,num_r,num_t,theta_offset,device).
    This avoids recomputing meshgrid/atan2 every forward.
    """
    def __init__(self):
        self._cache: Dict[Tuple, torch.Tensor] = {}

    @torch.no_grad()
    def get_joint(
        self,
        H: int,
        W: int,
        num_bins: int,
        num_theta_bins: int,
        theta_offset: float,
        device: torch.device,
    ) -> torch.Tensor:
        key = (H, W, num_bins, num_theta_bins, float(theta_offset), str(device))
        if key in self._cache:
            return self._cache[key]

        yy, xx = torch.meshgrid(
            torch.arange(H, device=device, dtype=torch.float32),
            torch.arange(W, device=device, dtype=torch.float32),
            indexing="ij",
        )
        cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
        dy = yy - cy
        dx = xx - cx

        # radius bins in [0, num_bins-1]
        r = torch.sqrt(dx * dx + dy * dy)
        r = r / (r.max().clamp_min(1e-12))
        r_bin = torch.clamp((r * (num_bins - 1)).to(torch.int64), 0, num_bins - 1)

        # theta bins in [0, num_theta_bins-1]
        theta = torch.atan2(dy, dx) + float(theta_offset)
        theta = (theta + math.pi) / (2.0 * math.pi)  # [0,1) with wrapping
        theta = theta - torch.floor(theta)
        t_bin = torch.clamp((theta * num_theta_bins).to(torch.int64), 0, num_theta_bins - 1)

        joint = (r_bin * num_theta_bins + t_bin).reshape(-1).contiguous()  # [H*W]
        self._cache[key] = joint
        return joint


# -------------------------
# ViT token grid inference
# -------------------------
def _infer_vit_grid(
    N: int,
    vit_grid: Optional[Tuple[int, int]],
    num_prefix_tokens: Optional[int],
    max_prefix_tokens: int,
) -> Tuple[int, int, int]:
    """
    Returns (H, W, num_prefix_tokens).
    """
    if vit_grid is not None:
        H, W = vit_grid
        needed = H * W
        if num_prefix_tokens is None:
            num_prefix_tokens = N - needed
        if N - num_prefix_tokens != needed:
            raise ValueError(
                f"vit_grid={vit_grid} implies {needed} patch tokens, but got N={N} "
                f"with num_prefix_tokens={num_prefix_tokens} => {N - num_prefix_tokens} tokens."
            )
        return H, W, num_prefix_tokens

    # infer square grid
    if num_prefix_tokens is None:
        found = None
        for k in range(0, max_prefix_tokens + 1):
            m = N - k
            if m <= 0:
                break
            s = int(math.isqrt(m))
            if s * s == m:
                found = k
                H = W = s
                break
        if found is None:
            raise ValueError(
                f"Cannot infer square token grid from N={N}. "
                f"Pass vit_grid=(H,W) or num_prefix_tokens explicitly."
            )
        return H, W, found

    m = N - num_prefix_tokens
    s = int(math.isqrt(m))
    if s * s != m:
        raise ValueError(
            f"N - num_prefix_tokens = {m} is not a perfect square. "
            f"Pass vit_grid=(H,W) instead."
        )
    return s, s, num_prefix_tokens


# -------------------------
# Differentiable polar spectrum (per-sample, not merged across batch)
# -------------------------
def polar_spectrum_2d(
    x: torch.Tensor,
    *,
    num_bins: int = 32,
    num_theta_bins: int = 64,
    eps: float = 1e-8,
    out_dtype=None,
    # ViT handling
    vit_grid: Optional[Tuple[int, int]] = None,
    num_prefix_tokens: Optional[int] = None,
    max_prefix_tokens: int = 16,
    use_cls_token: bool = True,
    # FFT/polar params
    remove_dc: bool = True,
    power: bool = True,
    theta_offset: float = 0.0,
    # output
    return_marginal_r: bool = False,
    # caching
    bin_cache: Optional[PolarBinCache] = None,
) -> torch.Tensor:
    """
    Inputs:
      - CNN map: x [B,C,H,W]
      - ViT tokens: x [B,N,D]
        * if use_cls_token=True: build 1-channel HxW map from CLS embedding by pad/truncation
        * else: reshape patch tokens to [B,D,H,W]

    Returns:
      polar: [B, R, Theta] distribution (sums to 1 per sample), differentiable wrt x
      optionally also returns marginal_r: [B, R]
    """
    if out_dtype is None:
        out_dtype = x.dtype

    # ---- convert ViT tokens -> [B,C,H,W] ----
    if x.dim() == 3:
        B, N, D = x.shape
        H, W, num_prefix_tokens = _infer_vit_grid(N, vit_grid, num_prefix_tokens, max_prefix_tokens)

        if use_cls_token:
            cls = x[:, 0, :]  # [B,D]
            HW = H * W
            if D < HW:
                cls = F.pad(cls, (0, HW - D), value=0.0)
            elif D > HW:
                cls = cls[:, :HW]
            x = cls.view(B, 1, H, W).contiguous()  # [B,1,H,W]
        else:
            tok = x[:, num_prefix_tokens:, :] if num_prefix_tokens > 0 else x
            tok = tok.contiguous().view(B, H, W, D).permute(0, 3, 1, 2).contiguous()  # [B,D,H,W]
            x = tok
    elif x.dim() != 4:
        raise ValueError(f"Expected x dim 3 or 4, got {x.dim()}")

    # ---- x: [B,C,H,W] -> grayscale [B,H,W] ----
    x = x.mean(dim=1)  # [B,H,W]
    B, H, W = x.shape

    # ---- FFT power spectrum ----
    X = torch.fft.fft2(x.to(torch.float32), dim=(-2, -1))
    X = torch.fft.fftshift(X, dim=(-2, -1))
    mag = X.abs()
    P = mag * mag if power else mag  # [B,H,W]

    if remove_dc:
        # avoid in-place on a view; use a multiplicative mask
        mask = torch.ones((H, W), device=P.device, dtype=P.dtype)
        mask[H // 2, W // 2] = 0.0
        P = P * mask

    # ---- joint bin indices ----
    if bin_cache is None:
        bin_cache = PolarBinCache()
    joint = bin_cache.get_joint(H, W, num_bins, num_theta_bins, theta_offset, P.device)  # [H*W]
    K = num_bins * num_theta_bins

    # ---- differentiable histogram per sample: histB [B,K] ----
    P_flat = P.reshape(B, -1)  # [B,HW]
    idx = joint.view(1, -1).expand(B, -1)  # view expand, no big allocation
    histB = torch.zeros((B, K), device=P.device, dtype=P.dtype)
    histB.scatter_add_(1, idx, P_flat)

    polar = histB.view(B, num_bins, num_theta_bins).to(torch.float32)
    polar = polar / (polar.sum(dim=(1, 2), keepdim=True) + eps)  # per-sample normalize

    if return_marginal_r:
        spectrum_r = polar.sum(dim=2)  # [B,R]
        spectrum_r = spectrum_r / (spectrum_r.sum(dim=1, keepdim=True) + eps)
        return polar.to(out_dtype), spectrum_r.to(out_dtype)

    return polar.to(out_dtype)


# -------------------------
# Losses (fast + strong gradients)
# -------------------------
def _power_temperature(p: torch.Tensor, T: float = 1.0, eps: float = 1e-8) -> torch.Tensor:
    """
    Temperature for probability distributions: p^(1/T) / Z
    (Much better behaved than softmax(log(p)/T) for your use case.)
    """
    p = p.to(torch.float32).clamp_min(eps)
    if T != 1.0:
        p = p.pow(1.0 / float(T))
    return p / (p.sum(dim=-1, keepdim=True) + eps)


def polar_kl_loss(
    polar_s: torch.Tensor,
    polar_t: torch.Tensor,
    *,
    T: float = 1.0,
    k_low: Optional[int] = 12,      # constrain only low radius bins (stronger + faster)
    eps: float = 1e-8,
    reduction: str = "mean",
    direction: str = "t||s",        # "t||s" => KL(target || student) (recommended)
) -> torch.Tensor:
    """
    polar_s: [B,R,Theta] or [R,Theta]
    polar_t: [B,R,Theta] or [R,Theta]
    Returns scalar.

    k_low:
      - if not None, only match polar[:, :k_low, :]
      - this massively speeds convergence
    """
    if polar_s.dim() == 2:
        polar_s = polar_s.unsqueeze(0)
    if polar_t.dim() == 2:
        polar_t = polar_t.unsqueeze(0)

    Bs = polar_s.shape[0]
    Bt = polar_t.shape[0]
    if Bt == 1 and Bs > 1:
        polar_t = polar_t.expand(Bs, -1, -1)
    elif Bt != Bs:
        raise ValueError(f"Batch mismatch: polar_s B={Bs}, polar_t B={Bt}")

    if k_low is not None:
        polar_s = polar_s[:, :k_low, :]
        polar_t = polar_t[:, :k_low, :]

    ps = polar_s.reshape(polar_s.shape[0], -1)
    pt = polar_t.reshape(polar_t.shape[0], -1)

    ps = ps / (ps.sum(dim=1, keepdim=True) + eps)
    pt = pt / (pt.sum(dim=1, keepdim=True) + eps)

    psT = _power_temperature(ps, T=T, eps=eps)
    ptT = _power_temperature(pt, T=T, eps=eps)

    if direction == "t||s":
        # KL(pt || ps)
        kl = (ptT * (ptT.clamp_min(eps).log() - psT.clamp_min(eps).log())).sum(dim=1)
    elif direction == "s||t":
        kl = (psT * (psT.clamp_min(eps).log() - ptT.clamp_min(eps).log())).sum(dim=1)
    else:
        raise ValueError('direction must be "t||s" or "s||t"')

    kl = (T * T) * kl
    if reduction == "mean":
        return kl.mean()
    if reduction == "sum":
        return kl.sum()
    return kl  # [B]


def lowfreq_profile_loss(
    polar_s,
    polar_t,
    *,
    k_low=12,
    eps=1e-8,
):
    ps_r = polar_s.sum(dim=2)[:, :k_low]
    pt_r = polar_t.sum(dim=2)[:, :k_low]

    ps_r = ps_r / (ps_r.sum(dim=1, keepdim=True) + eps)
    pt_r = pt_r / (pt_r.sum(dim=1, keepdim=True) + eps)

    return F.mse_loss(torch.log(ps_r + eps), torch.log(pt_r + eps))

: 

In [ ]:
import torch
import torch.nn.functional as F
import math


# ---------------------------
# Orthonormal DCT-II helpers
# ---------------------------

def _dct_1d_ortho(x: torch.Tensor) -> torch.Tensor:
    """
    DCT-II (ortho) along last dim.
    x: [..., N] real
    returns: [..., N] real
    """
    N = x.shape[-1]

    # even extension: [x0..xN-1, xN-1..x0]
    x_ext = torch.cat([x, x.flip(-1)], dim=-1)

    # FFT of even-extended signal
    X = torch.fft.fft(x_ext, dim=-1)  # complex

    k = torch.arange(N, device=x.device, dtype=torch.float32)
    W = torch.exp(-1j * math.pi * k / (2.0 * N))  # complex

    # DCT-II from FFT
    y = (X[..., :N] * W).real * 2.0

    # Orthonormal scaling
    y[..., 0] *= (1.0 / math.sqrt(4.0 * N))
    y[..., 1:] *= (1.0 / math.sqrt(2.0 * N))
    return y


def _dct_2d_ortho(x: torch.Tensor) -> torch.Tensor:
    """
    Separable 2D DCT-II (ortho).
    x: [*, H, W]
    returns: [*, H, W]
    """
    x = _dct_1d_ortho(x)  # along W
    x = _dct_1d_ortho(x.transpose(-1, -2)).transpose(-1, -2)  # along H
    return x


# ---------------------------
# Channelwise radial spectrum
# ---------------------------

def radial_spectrum_2d_channelwise_dct(
    x: torch.Tensor,
    num_bins: int = 16,
    eps: float = 1e-8,
    out_dtype=None,
    *,
    vit_grid=None,              # (H, W) explicit patch grid
    num_prefix_tokens=None,      # number of prefix tokens total
    max_prefix_tokens: int = 16,
    use_cls_token: bool = True,  # if True, build spectrum from CLS token only
    reduce: str = "mean",        # "mean" or "sum" across channels (and batch)
    use_log_power: bool = False, # log1p(power) to compress dynamic range
):
    """
    Supports:
      - CNN feature maps: x [B, C, H, W]
      - ViT tokens:       x [B, N, D]

    Returns:
      spectrum [num_bins], normalized to sum=1 (aggregated across batch and channels).
      (If you want per-channel spectra, see note at bottom.)
    """
    if out_dtype is None:
        out_dtype = x.dtype

    # ---------------- ViT handling (same spirit as your version) ----------------
    if x.dim() == 3:
        B, N, D = x.shape

        # Determine grid size and prefix tokens
        if vit_grid is not None:
            H, W = vit_grid
            needed = H * W
            if num_prefix_tokens is None:
                num_prefix_tokens = N - needed
            if N - num_prefix_tokens != needed:
                raise ValueError(
                    f"vit_grid={vit_grid} implies {needed} patch tokens, but got N={N} "
                    f"with num_prefix_tokens={num_prefix_tokens} => {N - num_prefix_tokens} tokens."
                )
        else:
            if num_prefix_tokens is None:
                found = None
                for k in range(0, max_prefix_tokens + 1):
                    m = N - k
                    if m <= 0:
                        break
                    s = int(math.isqrt(m))
                    if s * s == m:
                        found = k
                        H = W = s
                        break
                if found is None:
                    raise ValueError(
                        f"Cannot infer square token grid from N={N}. "
                        f"Pass vit_grid=(H,W) or num_prefix_tokens explicitly."
                    )
                num_prefix_tokens = found
            else:
                m = N - num_prefix_tokens
                s = int(math.isqrt(m))
                if s * s != m:
                    raise ValueError(
                        f"N - num_prefix_tokens = {m} is not a perfect square. "
                        f"Pass vit_grid=(H,W) instead."
                    )
                H = W = s

        if use_cls_token:
            # CLS assumed token 0: [B, D]
            cls = x[:, 0, :]
            HW = H * W
            if D < HW:
                cls = F.pad(cls, (0, HW - D), mode="constant", value=0.0)
            elif D > HW:
                cls = cls[:, :HW]
            # [B, H, W] -> [B, 1, H, W]
            x = cls.view(B, H, W).unsqueeze(1).contiguous()
        else:
            # Patch tokens as channels (D channels over HxW)
            if num_prefix_tokens > 0:
                x = x[:, num_prefix_tokens:, :].contiguous()  # [B, H*W, D]
            else:
                x = x.contiguous()
            x = x.view(B, H, W, D).permute(0, 3, 1, 2).contiguous()  # [B, D, H, W]

    elif x.dim() != 4:
        raise ValueError(f"Expected x dim 3 or 4, got {x.dim()}")

    # x is now [B, C, H, W]
    B, C, H, W = x.shape

    # ---------------- DCT power (channelwise) ----------------
    # do DCT in fp32 for stability
    xf = x.to(torch.float32)

    # compute 2D DCT for each (B,C) map: reshape to [B*C, H, W]
    X = xf.reshape(B * C, H, W)
    DCT = _dct_2d_ortho(X)  # [B*C, H, W]

    P = DCT * DCT  # power, [B*C, H, W]
    if use_log_power:
        P = torch.log1p(P)

    # ---------------- Radial bins (DCT: DC at (0,0), HF grows with radius) ----------------
    yy, xx = torch.meshgrid(
        torch.arange(H, device=P.device),
        torch.arange(W, device=P.device),
        indexing="ij",
    )
    r = torch.sqrt(yy.to(torch.float32) ** 2 + xx.to(torch.float32) ** 2)  # [H,W]
    r = r / (r.max() + 1e-12)
    rbin = (r * (num_bins - 1)).to(torch.long)  # [H,W] in [0, num_bins-1]

    # ---------------- Accumulate spectrum ----------------
    # Flatten spatial dims and scatter-add across all (B*C) maps.
    idx = rbin.flatten().unsqueeze(0).expand(B * C, -1)  # [BC, HW]
    vals = P.reshape(B * C, -1)                          # [BC, HW]

    spec = torch.zeros(num_bins, device=P.device, dtype=torch.float32)
    spec.scatter_add_(0, idx.reshape(-1), vals.reshape(-1))

    # Optional: if you want to reduce per (B,C) first, you can do it differently,
    # but this global accumulation is fastest and matches your original behavior.

    if reduce not in ("mean", "sum"):
        raise ValueError(f"reduce must be 'mean' or 'sum', got {reduce}")

    if reduce == "mean":
        spec = spec / float(B * C)

    spec = spec / (spec.sum() + eps)
    return spec.to(out_dtype)


: 

In [ ]:
import math
import torch

def radial_spectrum_2d(
    x,
    num_bins=16,
    eps=1e-8,
    out_dtype=None,

    *,
    vit_grid=None,              # (H, W) explicit patch grid
    num_prefix_tokens=None,      # number of prefix tokens total
    max_prefix_tokens=16,
    use_cls_token=True,         # NEW: if True, build spectrum from CLS token only
):
    """
    Supports:
      - CNN feature maps: x [B, C, H, W]
      - ViT tokens:       x [B, N, D]  (tokens, embed dim)

    Returns:
      spectrum [num_bins], normalized to sum=1 (aggregated across batch).
    """

    if out_dtype is None:
        out_dtype = x.dtype

    if x.dim() == 3:
        # ViT tokens: [B, N, D]
        B, N, D = x.shape

        # Determine grid size and how many prefix tokens exist.
        if vit_grid is not None:
            H, W = vit_grid
            needed = H * W
            if num_prefix_tokens is None:
                num_prefix_tokens = N - needed
            if N - num_prefix_tokens != needed:
                raise ValueError(
                    f"vit_grid={vit_grid} implies {needed} patch tokens, but got N={N} "
                    f"with num_prefix_tokens={num_prefix_tokens} => {N - num_prefix_tokens} tokens."
                )
        else:
            # Auto-infer num_prefix_tokens by finding N-k that is a perfect square
            if num_prefix_tokens is None:
                found = None
                for k in range(0, max_prefix_tokens + 1):
                    m = N - k
                    if m <= 0:
                        break
                    s = int(math.isqrt(m))
                    if s * s == m:
                        found = k
                        H = W = s
                        break
                if found is None:
                    raise ValueError(
                        f"Cannot infer square token grid from N={N}. "
                        f"Pass vit_grid=(H,W) or num_prefix_tokens explicitly."
                    )
                num_prefix_tokens = found
            else:
                m = N - num_prefix_tokens
                s = int(math.isqrt(m))
                if s * s != m:
                    raise ValueError(
                        f"N - num_prefix_tokens = {m} is not a perfect square. "
                        f"Pass vit_grid=(H,W) instead."
                    )
                H = W = s

        # =========================
        # NEW: CLS-token spectrum
        # =========================
        if use_cls_token:
            # CLS assumed to be token 0
            cls = x[:, 0, :]  # [B, D]

            # Turn CLS embedding into a 2D "image" so FFT makes sense.
            # We reshape D -> H*W (requires D == H*W).
            # If not equal, we crop or pad.
            HW = H * W
            if D < HW:
                pad = HW - D
                cls = torch.nn.functional.pad(cls, (0, pad), mode="constant", value=0.0)
            elif D > HW:
                cls = cls[:, :HW]

            # [B, HW] -> [B, H, W] -> [B, 1, H, W]
            x = cls.view(B, H, W).unsqueeze(1).contiguous()

        else:
            # Default: patch-token spectrum (original behavior)
            if num_prefix_tokens > 0:
                x = x[:, num_prefix_tokens:, :].contiguous()  # [B, H*W, D]
            else:
                x = x.contiguous()

            # [B, H*W, D] -> [B, H, W, D] -> [B, D, H, W]
            x = x.view(B, H, W, D).permute(0, 3, 1, 2).contiguous()

    elif x.dim() != 4:
        raise ValueError(f"Expected x dim 3 or 4, got {x.dim()}")
    
    

    # CNN path (or converted ViT path): x [B, C, H, W]
    x = x.mean(dim=1)                 # [B, H, W]

    # FFT in fp32 for safety/stability
    x_fft = x.to(torch.float32)
    F = torch.fft.fft2(x_fft)         # complex64
    F = torch.fft.fftshift(F)  # shift only spatial dims

    P = (F.real * F.real + F.imag * F.imag)  # [B, H, W] float32

    B, H, W = P.shape
    yy, xx = torch.meshgrid(
        torch.arange(H, device=P.device),
        torch.arange(W, device=P.device),
        indexing="ij"
    )
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    r = torch.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r = (r / (r.max() + 1e-12) * (num_bins - 1)).to(torch.long)  # [H, W]

    spectrum = torch.zeros(num_bins, device=P.device, dtype=torch.float32)
    idx = r.flatten().expand(B, -1)     # [B, HW]
    vals = P.reshape(B, -1)            # [B, HW]
    spectrum.scatter_add_(0, idx.reshape(-1), vals.reshape(-1))

    return spectrum.to(out_dtype)



@torch.compile(fullgraph=True)
def wasserstein_1d(p, q, eps=1e-8):
    """
    p, q : tensors [K]
    """
    p32 = p.to(torch.float32)
    q32 = q.to(torch.float32)

    p32 = p32 / (p32.sum() + eps)
    q32 = q32 / (q32.sum() + eps)

    cdf_p = torch.cumsum(p32, dim=0)
    cdf_q = torch.cumsum(q32, dim=0)

    return torch.abs(cdf_p - cdf_q).to(p.dtype)


def act_freq_loss(model_act, target_image, num_bins=16):
    model_spec = radial_spectrum_2d(model_act, num_bins=num_bins)
    target_spec = radial_spectrum_2d(target_image, num_bins=num_bins)

    delta = wasserstein_1d(model_spec, target_spec)

    # with torch.no_grad():
    gamma = 1.5
    z = delta / (delta.mean() + 1e-6)
    w = z.pow(gamma).clamp(0.2, 10.0)   # gamma=1.5~2
    # w = torch.softmax(delta / 1.0, dim=0)


    return (w * delta).mean()






: 

In [ ]:
import math
import torch
def radial_spectrum_2d(
    x,
    num_bins=16,
    eps=1e-8,
    out_dtype=None,

    *,
    vit_grid=None,              # (H, W) explicit patch grid
    num_prefix_tokens=None,      # number of prefix tokens total
    max_prefix_tokens=16,
    use_cls_token=True,         # NEW: if True, build spectrum from CLS token only
):
    """
    Supports:
      - CNN feature maps: x [B, C, H, W]
      - ViT tokens:       x [B, N, D]  (tokens, embed dim)

    Returns:
      spectrum [num_bins], normalized to sum=1 (aggregated across batch).
    """

    if out_dtype is None:
        out_dtype = x.dtype

    if x.dim() == 3:
        # ViT tokens: [B, N, D]
        B, N, D = x.shape

        # Determine grid size and how many prefix tokens exist.
        if vit_grid is not None:
            H, W = vit_grid
            needed = H * W
            if num_prefix_tokens is None:
                num_prefix_tokens = N - needed
            if N - num_prefix_tokens != needed:
                raise ValueError(
                    f"vit_grid={vit_grid} implies {needed} patch tokens, but got N={N} "
                    f"with num_prefix_tokens={num_prefix_tokens} => {N - num_prefix_tokens} tokens."
                )
        else:
            # Auto-infer num_prefix_tokens by finding N-k that is a perfect square
            if num_prefix_tokens is None:
                found = None
                for k in range(0, max_prefix_tokens + 1):
                    m = N - k
                    if m <= 0:
                        break
                    s = int(math.isqrt(m))
                    if s * s == m:
                        found = k
                        H = W = s
                        break
                if found is None:
                    raise ValueError(
                        f"Cannot infer square token grid from N={N}. "
                        f"Pass vit_grid=(H,W) or num_prefix_tokens explicitly."
                    )
                num_prefix_tokens = found
            else:
                m = N - num_prefix_tokens
                s = int(math.isqrt(m))
                if s * s != m:
                    raise ValueError(
                        f"N - num_prefix_tokens = {m} is not a perfect square. "
                        f"Pass vit_grid=(H,W) instead."
                    )
                H = W = s

        # =========================
        # NEW: CLS-token spectrum
        # =========================
        if use_cls_token:
            # CLS assumed to be token 0
            cls = x[:, 0, :]  # [B, D]

            # Turn CLS embedding into a 2D "image" so FFT makes sense.
            # We reshape D -> H*W (requires D == H*W).
            # If not equal, we crop or pad.
            HW = H * W
            if D < HW:
                pad = HW - D
                cls = torch.nn.functional.pad(cls, (0, pad), mode="constant", value=0.0)
            elif D > HW:
                cls = cls[:, :HW]

            # [B, HW] -> [B, H, W] -> [B, 1, H, W]
            x = cls.view(B, H, W).unsqueeze(1).contiguous()

        else:
            # Default: patch-token spectrum (original behavior)
            if num_prefix_tokens > 0:
                x = x[:, num_prefix_tokens:, :].contiguous()  # [B, H*W, D]
            else:
                x = x.contiguous()

            # [B, H*W, D] -> [B, H, W, D] -> [B, D, H, W]
            x = x.view(B, H, W, D).permute(0, 3, 1, 2).contiguous()

    elif x.dim() != 4:
        raise ValueError(f"Expected x dim 3 or 4, got {x.dim()}")
    
    

    # CNN path (or converted ViT path): x [B, C, H, W]
    x = x.mean(dim=1)                 # [B, H, W]

    # FFT in fp32 for safety/stability
    x_fft = x.to(torch.float32)
    F = torch.fft.fft2(x_fft)         # complex64
    F = torch.fft.fftshift(F)  # shift only spatial dims

    P = (F.real * F.real + F.imag * F.imag)  # [B, H, W] float32

    B, H, W = P.shape
    yy, xx = torch.meshgrid(
        torch.arange(H, device=P.device),
        torch.arange(W, device=P.device),
        indexing="ij"
    )
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    r = torch.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r = (r / (r.max() + 1e-12) * (num_bins - 1)).to(torch.long)  # [H, W]

    spectrum = torch.zeros(num_bins, device=P.device, dtype=torch.float32)
    idx = r.flatten().expand(B, -1)     # [B, HW]
    vals = P.reshape(B, -1)            # [B, HW]
    spectrum.scatter_add_(0, idx.reshape(-1), vals.reshape(-1))

    spectrum = spectrum / (spectrum.sum() + eps)
    return spectrum.to(out_dtype)
# import torch
# import math
# import torch.nn.functional as F


# # ---------------------------------------------------
# # DCT-II (orthonormal), separable 2D implementation
# # ---------------------------------------------------

# def dct_1d(x, norm="ortho"):
#     """
#     x: [..., N]
#     returns DCT-II along last dim
#     """
#     N = x.shape[-1]

#     # Even-symmetric extension
#     x_ext = torch.cat([x, x.flip(-1)], dim=-1)

#     # FFT
#     X = torch.fft.fft(x_ext, dim=-1)

#     # Take real cosine part
#     k = torch.arange(N, device=x.device)
#     W = torch.exp(-1j * math.pi * k / (2 * N))

#     out = (X[..., :N] * W).real * 2

#     if norm == "ortho":
#         out[..., 0] /= math.sqrt(4 * N)
#         out[..., 1:] /= math.sqrt(2 * N)
#     return out


# def dct_2d(x):
#     """
#     x: [B,H,W]
#     """
#     x = dct_1d(x, norm="ortho")
#     x = dct_1d(x.transpose(-1, -2), norm="ortho").transpose(-1, -2)
#     return x


# # ---------------------------------------------------
# # Radial DCT spectrum (your main function)
# # ---------------------------------------------------

# def radial_spectrum_2d(
#     x,
#     num_bins=16,
#     eps=1e-8,
#     out_dtype=None,

#     *,
#     vit_grid=None,
#     num_prefix_tokens=None,
#     max_prefix_tokens=16,
#     use_cls_token=True,
# ):
#     """
#     Supports:
#       - CNN feature maps: x [B,C,H,W]
#       - ViT tokens:       x [B,N,D]

#     Returns:
#       spectrum [num_bins], normalized to sum=1
#     """

#     if out_dtype is None:
#         out_dtype = x.dtype

#     # ---------------- ViT handling ----------------

#     if x.dim() == 3:
#         B, N, D = x.shape

#         if vit_grid is not None:
#             H, W = vit_grid
#             needed = H * W
#             if num_prefix_tokens is None:
#                 num_prefix_tokens = N - needed
#         else:
#             if num_prefix_tokens is None:
#                 for k in range(max_prefix_tokens + 1):
#                     m = N - k
#                     s = int(math.isqrt(m))
#                     if s * s == m:
#                         num_prefix_tokens = k
#                         H = W = s
#                         break
#             else:
#                 m = N - num_prefix_tokens
#                 s = int(math.isqrt(m))
#                 H = W = s

#         if use_cls_token:
#             cls = x[:, 0, :]  # [B,D]
#             HW = H * W
#             if D < HW:
#                 cls = F.pad(cls, (0, HW - D))
#             elif D > HW:
#                 cls = cls[:, :HW]

#             x = cls.view(B, H, W).unsqueeze(1)

#         else:
#             if num_prefix_tokens > 0:
#                 x = x[:, num_prefix_tokens:, :]
#             x = x.view(B, H, W, D).permute(0, 3, 1, 2)

#     elif x.dim() != 4:
#         raise ValueError(f"Expected x dim 3 or 4, got {x.dim()}")

#     # ------------------------------------------------
#     # CNN path: x [B,C,H,W] → spatial mean
#     # ------------------------------------------------

#     x = x.mean(dim=1)   # [B,H,W]

#     # ------------------------------------------------
#     # DCT (fp32 for stability)
#     # ------------------------------------------------

#     x = x.float()
#     F = dct_2d(x)
#     P = F.pow(2)        # power spectrum [B,H,W]

#     B, H, W = P.shape

#     # ------------------------------------------------
#     # Radial bins
#     # ------------------------------------------------

#     yy, xx = torch.meshgrid(
#         torch.arange(H, device=P.device),
#         torch.arange(W, device=P.device),
#         indexing="ij"
#     )

#     r = torch.sqrt(yy.float() ** 2 + xx.float() ** 2)
#     r = r / (r.max() + 1e-12)
#     r = (r * (num_bins - 1)).long()

#     spectrum = torch.zeros(num_bins, device=P.device)

#     idx = r.flatten().expand(B, -1)
#     vals = P.reshape(B, -1)

#     spectrum.scatter_add_(0, idx.reshape(-1), vals.reshape(-1))

#     spectrum = spectrum / (spectrum.sum() + eps)

#     return spectrum.to(out_dtype)


@torch.compile(fullgraph=True)
def wasserstein_1d(p, q, eps=1e-8):
    """
    p, q : tensors [K]
    """
    p32 = p.to(torch.float32)
    q32 = q.to(torch.float32)

    p32 = p32 / (p32.sum() + eps)
    q32 = q32 / (q32.sum() + eps)

    cdf_p = torch.cumsum(p32, dim=0)
    cdf_q = torch.cumsum(q32, dim=0)

    return torch.sum(torch.abs(cdf_p - cdf_q)).to(p.dtype)

@torch.compile(fullgraph=True)
def kl_div_spectrum(q, p, eps=1e-8):
    # q, p: [B,K], each row sums to 1
    q = q.clamp_min(eps)
    p = p.clamp_min(eps)
    return (q * (q.log() - p.log())).sum(dim=-1).mean()


@torch.compile(fullgraph=True)
def temp_kl(p, q, T=2.0, eps=1e-8):
    """
    p, q: [B, K] distributions (sum=1)
    Applies temperature to log-prob space:
      softmax(log(p)/T)
    """
    logp = torch.log(p.clamp_min(eps))
    logq = torch.log(q.clamp_min(eps))

    pT = F.softmax(logp / T, dim=-1)
    qT = F.softmax(logq / T, dim=-1)

    # KL(p||q) = sum p * (log p - log q)
    kl = (pT * (torch.log(pT.clamp_min(eps)) - torch.log(qT.clamp_min(eps)))).sum(dim=-1).mean()

    return (T * T) * kl



: 

In [ ]:
activation_cache.keys()

: 

In [ ]:
model_spectrum = radial_spectrum_2d(activation_cache['layer4.1.conv2'].to(torch.float32))
target_spectrum = radial_spectrum_2d(x.to(torch.float32)).cpu()

: 

In [ ]:
wasserstein_1d(model_spectrum, target_spectrum)

: 

In [ ]:
for n, m in model.named_modules():
    if isinstance(m, (nn.Linear, nn.Conv1d, nn.Conv2d)):
        print(n)

: 

In [ ]:
model.eval()
activation_cache = {}
low_frequency_energy_ratio_grad = {}
low_frequency_energy_ratio_act = {}
baseline = copy.deepcopy(model.state_dict())

hook_handles = {}
cahced_init = None
cached_rng_state = None

def kaiming_normal_fan_in(w):
    return nn.init.kaiming_normal_(w, mode='fan_in', nonlinearity='relu')
def kaiming_normal_fan_out(w):
    return nn.init.kaiming_normal_(w, mode='fan_out', nonlinearity='relu')
def kaiming_uniform_fan_in(w):
    return nn.init.kaiming_uniform_(w, mode='fan_in', nonlinearity='relu')
def kaiming_uniform_fan_out(w):
    return nn.init.kaiming_uniform_(w, mode='fan_out', nonlinearity='relu')
def trunc_normal(w):
    return trunc_normal_(w, std=0.02)
def trunc_normal_small(w):
    return trunc_normal_(w, std=0.01)
def orthogonal(w):
    return nn.init.orthogonal_(w, gain=1.0)
def orthogonal2(w):
    return nn.init.orthogonal_(w, gain=math.sqrt(2))
def xavier_normal(w):
    return nn.init.xavier_normal_(w)
def xavier_uniform(w):
    return nn.init.xavier_uniform_(w)
def lecun_normal(w):
    return nn.init.normal_(w, std=1.0 / math.sqrt(w.shape[1]))
def kaiming_low_std(w):
    fan_in = nn.init._calculate_correct_fan(w, "fan_in")
    std = math.sqrt(2.0 / fan_in) * 0.5
    return nn.init.normal_(w, std=std)


def forward_hook(name):
    def hook(module, input, output):
        activation_cache[name] = output.detach()
    return hook


min_score = 100
for init in [
    kaiming_normal_fan_in,
    kaiming_normal_fan_out,
    kaiming_uniform_fan_in,
    kaiming_uniform_fan_out,
    kaiming_low_std,
    trunc_normal,
    trunc_normal_small,
    orthogonal,
    orthogonal2,
    xavier_normal,
    xavier_uniform,
    lecun_normal
    ]:

    # 42, 49, 0, 1, 2, 3, 4
    set_seed(49)
    cpu_rng_state = torch.get_rng_state()
    gpu_rng_state = torch.cuda.get_rng_state()

    for m in model.modules():
        if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Linear)):
            init(m.weight)
    print(init)

    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv1d, nn.Conv2d, nn.Linear)):
            hook_handles[name] = module.register_forward_hook(forward_hook(name))

    with torch.no_grad():
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            logits = model(x)
            loss = criterion(logits, y)

    act = activation_cache['features.8.0']

    model_spectrum = radial_spectrum_2d(act.to(torch.float32), num_bins=16).cpu()
    target_spectrum = radial_spectrum_2d(x.to(torch.float32), num_bins=16).cpu()

    score = wasserstein_1d(model_spectrum, target_spectrum)
    if score < min_score:
        min_score = score
        cahced_init = init
        cached_rng_state = {
            'CPU' : cpu_rng_state,
            'GPU' : gpu_rng_state
        }
        print(f"min score: {min_score}")


    for h in hook_handles.values():
        h.remove()
    activation_cache.clear()
    torch.cuda.empty_cache()



: 

In [ ]:
cahced_init

: 

In [ ]:
torch.cuda.empty_cache()

: 

In [ ]:
#torch vision default
tensor(17.0655, device='cuda:0')

# timm's trunc_init
tensor(25.9191, device='cuda:0')

# fan out
tensor(11.3065, device='cuda:0')

# init_orthogonal_conv(m, gain=math.sqrt(2))
tensor(22.3052, device='cuda:0')

# init_orthogonal_conv(m, gain=1.0)
tensor(7.9659, device='cuda:0')

: 

In [ ]:
x, y = next(iter(train_loader))
x, y = x.to('cuda'), y.to('cuda')

: 

In [ ]:
with profile(
    activities=[
        ProfilerActivity.CPU,
        ProfilerActivity.CUDA
    ],
    record_shapes=True,
    with_stack=True,
    profile_memory=True,
) as prof:

    for step in range(10):
        with record_function("train_step"):
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                output = controller.traced_model(x)
                loss = criterion(output, y)
                loss.backward()

torch.cuda.synchronize()

print(prof.key_averages().table(
    sort_by="cuda_time_total",
    row_limit=200
))


: 